# 🚨🚨🚨 LEIA-ME 🚨🚨🚨

Se estiver utilizando o Google Colab.

Antes de iniciar a execução do código, é importante alterar a versão do runtime para a que possui o Python na versão 3.11.*, que atualmente segue sendo a versão "2025.07".

Logo abaixo mostro como alterar a versão do Runtime.

![Acesse o Runtime](https://i.imgur.com/EOQ8KvA.png)

Clique no menu Runtime (Ambiente de execução). Vai abrir um dropdown, nele você deve clicar na opção "Change runtime type" (Trocar tipo de ambiente de execução).

![Trocar tipo do Runtime](https://i.imgur.com/XYCBjPZ.png)

Vai abrir um modal, nele você deve apenas selecionar a opção no select de "Runtime version" (Versões do ambiente de execução) que tenha o nome "2025.07".

Olhando na [documentação](https://research.google.com/colaboratory/runtime-version-faq.html#2025.07) você pode observar que a versão do python é a que necessitamos (3.11.*).

![Alterar versão](https://i.imgur.com/kz9XhDD.png)

O motivo da troca de versão é porque temos dependências que não foram disponiilizadas para as versões mais recentes do python.

![]()

# Informações sobre o projeto

## Integrantes

Os integrantes desse projeto são:
- [Rhogger Freitas Silva](https://www.linkedin.com/in/rhogger-fs/)
- [José Henrique Queiroz de Souza](https://www.linkedin.com/in/josehenriqve/)
- [Mateus Abreu da Cunha Nascimento](https://www.linkedin.com/in/mateusabreucn/)
- [Felipe Peretti](https://www.linkedin.com/in/felipeperetti/)

## Sobre o projeto

Este é um projeto desenvolvido durante a disciplina "Projeto de Ciência de Dados" da pós-graduação de Data Science & Machine Learning da UniRV.

<br>

O foco da disciplina é criar um projeto completo de ciência de dados para aplicarmos os conhecimentos obtidos ao longo do curso e apresentá-lo à uma banca avaliadora composta de profissionais das áreas na qual foi aplicado o projeto.

<br>

O objetivo do projeto é limpar e preparar os dados para extração de insights, preprocessar e normalizar os dados para a geração de modelos na predição se uma determinada ação ordinária da bolsa de valores é recomendado o investimento, e estimar o seu valor em 2025.

<br>

Fonte dos dados: [Economatica](https://www.economatica.com/)

# Configuração de ambientes

## Instalação de dependências

Essa etapa demora um pouco, por favor aguarde.

### 🚨🚨🚨 Após finalizar, reinicie a sessão 🚨🚨🚨

In [ ]:
import os

# Primeiro, tenta carregar as variáveis do .env se a biblioteca estiver disponível
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("python-dotenv não está instalado. Tentando verificar através de outras formas...")

# Verifica se está rodando no Google Colab
# Prioriza a variável GOOGLE_COLAB_RUNTIME do .env, mas também verifica 'google.colab' no sys.modules
is_colab = False

# Primeiro verifica a variável de ambiente
google_colab_env = os.getenv('GOOGLE_COLAB_RUNTIME')
print(f"Valor da variável GOOGLE_COLAB_RUNTIME: {google_colab_env}")

if google_colab_env and google_colab_env.lower() == 'true':
    is_colab = True
    print("Google Colab detectado através da variável GOOGLE_COLAB_RUNTIME")
else:
    # Fallback: verifica se está rodando no Google Colab através do sys.modules
    import sys
    if 'google.colab' in sys.modules:
        is_colab = True
        print("Google Colab detectado através do módulo google.colab")

if is_colab:
    print("Executando no Google Colab - Instalando dependências...")
    %pip install -qqq python-dotenv
    %pip uninstall -qqq numpy pandas pycaret -y
    %pip install -qqq numpy pandas pycaret[full] category_encoders
else:
    print("Não está no Google Colab - Pulando instalação de dependências via pip")

## Importações

In [ ]:
import sys
print(sys.version)

In [ ]:
import pandas as pd
import numpy as np

import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go

import gdown

from sklearn.preprocessing import LabelEncoder
from scipy import stats

from pycaret.regression import *
from pycaret.classification import *
import category_encoders as ce

import re
import unicodedata
from collections import defaultdict


## Download de Datasets

Nessa etapa realizamos o download dos datasets (em formato csv) e armazenamento local aqui no colab, utilizando a biblioteca `gdown`, para não se fazer necessário o upload manual por cada usuário deste notebook.

Este primeiro dataset, é o que possui as principais informações que iremos utilizar.

In [ ]:
# Criar a pasta datasets se não existir (apenas no ambiente local)
if not is_colab:
    os.makedirs('../datasets', exist_ok=True)

url = 'https://drive.google.com/file/d/1XmkjSrATB20AafqaxJb6dU-4NGllGcPt/view'

# Define o caminho baseado no ambiente
if is_colab:
    output = '/content/df_economatica.csv'
else:
    output = '../datasets/df_economatica.csv'

print(f"Baixando arquivo para: {output}")
gdown.download(url, output, fuzzy=True)

Já este segundo, é o outro que ficou faltando algumas colunas relevantes para a nossa análise posterior.

In [ ]:
url = 'https://drive.google.com/file/d/1FyURuDTdL-hVONTWO2y9rTaUfz88Fmyj/view'

# Define o caminho baseado no ambiente
if is_colab:
    output = '/content/df_economaticav2.csv'
else:
    output = '../datasets/df_economaticav2.csv'

print(f"Baixando arquivo para: {output}")
gdown.download(url, output, fuzzy=True)

## Configurações Pandas

Configurações de exibição do pandas atualizadas para mostrar todas as linhas e colunas, pois o dataset possui mais de 100 colunas, logo o pandas trunca as tabelas.

In [ ]:
if is_colab:
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)

## Configurações Plotly

In [ ]:
# Cores corrigidas conforme especificado
GRADIO_PRIMARY = "#F97316"       # Cor primária (laranja)
GRADIO_SECONDARY = "#3267BD"     # Cor secundária (azul)
GRADIO_DARK_BG = "#27272A"       # Fundo do gráfico
GRADIO_CARD_BG = "#27272A"       # Fundo dos cards/containers (mesmo que o bg do gráfico)
GRADIO_BORDER = "#3F3F46"        # Linhas do gráfico/bordas
GRADIO_TEXT = "#E7E7E8"          # Labels/texto principal
GRADIO_TEXT_MUTED = "#A1A1AA"    # Texto secundário (derivado dos labels)

# Criar tema customizado baseado no gradio dark com cores corretas
custom_theme = {
    "layout": {
        "paper_bgcolor": GRADIO_DARK_BG,     # Fundo principal do gráfico
        "plot_bgcolor": GRADIO_DARK_BG,      # Fundo da área de plotagem
        "font": {"color": GRADIO_TEXT, "family": "Inter, system-ui, sans-serif"},
        "colorway": [GRADIO_PRIMARY, GRADIO_SECONDARY, "#10b981", "#f59e0b", "#ef4444", "#8b5cf6", "#06b6d4", "#84cc16"],
        "title": {
            "font": {"color": GRADIO_TEXT, "size": 18},
            "x": 0.5,  # Centraliza título
            "xanchor": "center"
        },
        "xaxis": {
            "gridcolor": GRADIO_BORDER,
            "linecolor": GRADIO_BORDER,
            "tickcolor": GRADIO_BORDER,
            "color": GRADIO_TEXT,
            "zerolinecolor": GRADIO_BORDER
        },
        "yaxis": {
            "gridcolor": GRADIO_BORDER, 
            "linecolor": GRADIO_BORDER,
            "tickcolor": GRADIO_BORDER,
            "color": GRADIO_TEXT,
            "zerolinecolor": GRADIO_BORDER
        },
        "legend": {
            "bgcolor": f"rgba({int(GRADIO_CARD_BG[1:3], 16)}, {int(GRADIO_CARD_BG[3:5], 16)}, {int(GRADIO_CARD_BG[5:7], 16)}, 0.9)",
            "bordercolor": GRADIO_BORDER,
            "font": {"color": GRADIO_TEXT}
        },
        "coloraxis": {
            "colorbar": {
                "tickcolor": GRADIO_BORDER,
                "title": {"font": {"color": GRADIO_TEXT}}
            }
        }
    }
}

# Registrar o tema customizado
pio.templates["gradio_dark"] = go.layout.Template(custom_theme)

# Definir como tema padrão
pio.templates.default = "gradio_dark"

## Definição de datasets

Agora iremos armazenar os valores dos datasets à variáveis do tipo `DataFrame` do `pandas`, no qual iremos utilizaremos ao decorrer do projeto.

In [ ]:
if is_colab:
    path_main = '/content/df_economatica.csv'
    path_dividendos = '/content/df_economaticav2.csv'
else:
    path_main = '../datasets/df_economatica.csv'
    path_dividendos = '../datasets/df_economaticav2.csv'

df_economatica = pd.read_csv(path_main)
df_economatica_dividendos = pd.read_csv(path_dividendos)

In [ ]:
df_economatica.info()

In [ ]:
df_economatica.shape

Aqui confirmamos a quantidade de linhas do dataset para uma mesclagem dosdatasets, posteriormente. Este valor é importante.

In [ ]:
df_economatica.head()

In [ ]:
df_economatica_dividendos.info()

In [ ]:
df_economatica_dividendos.shape

Já aqui vemos que não há a mesma quantidade de linhas. Ou seja, iremos analisar e prorrogar a mesclagem.

In [ ]:
df_economatica_dividendos.head()

# Manipulação de features

## Filtrando a classe por ações ordinárias (ON)

In [ ]:
df_economatica_on = df_economatica[df_economatica['Classe'] == 'ON']

df_economatica_on.shape

In [ ]:
df_economatica_dividendos_on = df_economatica_dividendos[df_economatica_dividendos['Classe'] == 'ON']

df_economatica_dividendos_on.shape

## Remoção de features

Removendo as colunas "Classe", "Bolsa / Fonte", "Tipo de Ativo", "Ativo / Cancelado" e "Setor Agro Bovespa", pois todas essas colunas possuíam o mesmo valor para todas as linhas (ações), sendo "ON", "Bovespa", "Ação" e "ativo", respectivamente, esses valores não serão relevantes a partir de agora e todos possuem o mesmo valor.

<br>

A justificativa de remover a coluna "Setor Agro Bovespa" se dá pelo fato de que iremos realizar o OneHotEndcoding mais para frente, logo é mais simples remover a coluna do que normalizar os valores.

<br>

As colunas de ativos/passivos circulantes e não circulantes, são os componentes que formam valores maiores, como o ativo_total e o passivo_total. A informação contida nelas já está representada de forma mais consolidada e útil nos próprios totais e, principalmente, nos indicadores de liquidez e endividamento. Mantê-las no modelo criaria uma redundância desnecessária. Porém elas serão removidas posteriormente, pois serão utilizadas para cálculos.

In [ ]:
colunas_para_remover_df_economatica = [
    'Classe',
    'Bolsa / Fonte',
    'Tipo de Ativo',
    'Ativo /\nCancelado',
    'Setor Agro\nBovespa',
    # 'AtvCir\n < Dez 2020\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvCir\n < Dez 2021\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvCir\n < Dez 2022\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvCir\n < Dez 2023\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvCir\n < Dez 2024\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvNaoCir\n < Dez 2020\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvNaoCir\n < Dez 2021\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvNaoCir\n < Dez 2022\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvNaoCir\n < Dez 2023\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'AtvNaoCir\n < Dez 2024\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasCir\n < Dez 2020\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasCir\n < Set 2021\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasCir\n < Dez 2022\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasCir\n < Dez 2023\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasCir\n < Dez 2024\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasNoCir\n < Dez 2020\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasNoCir\n < Dez 2021\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasNoCir\n < Dez 2022\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasNoCir\n < Dez 2023\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'PasNoCir\n < Dez 2024\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Ativo Tot\n < Dez 2020\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Ativo Tot\n < Dez 2021\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Ativo Tot\n < Dez 2022\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Ativo Tot\n < Dez 2023\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Ativo Tot\n < Dez 2024\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Patrim Liq\n < Dez 2020\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Patrim Liq\n < Dez 2021\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Patrim Liq\n < Dez 2022\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Patrim Liq\n < Dez 2023\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Patrim Liq\n < Dez 2024\n Em moeda orig\n em milhares\n consolid:sim*',
    # 'Receita\n < Dez 2020\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Receita\n < Dez 2021\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Receita\n < Dez 2022\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Receita\n < Dez 2023\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Receita\n < Dez 2024\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Lucro Liquido\n < Dez 2020\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Lucro Liquido\n < Dez 2021\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Lucro Liquido\n < Dez 2022\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Lucro Liquido\n < Dez 2023\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
    # 'Lucro Liquido\n < Dez 2024\n Em moeda orig\n em milhares\n no exercício\n consolid:sim*',
]

df_economatica_colunas_removidas = df_economatica_on.drop(columns=colunas_para_remover_df_economatica).copy()

df_economatica_colunas_removidas.shape

In [ ]:
colunas_para_remover_df_economatica_dividendos = [
    'Classe',
    'Bolsa / Fonte',
    'Tipo de Ativo',
    'Ativo /\nCancelado'
]

df_economatica_dividendos_colunas_removidas = df_economatica_dividendos_on.drop(columns=colunas_para_remover_df_economatica_dividendos)

df_economatica_dividendos_colunas_removidas.shape

## Renomeando as colunas

In [ ]:
novos_nomes_colunas_df_economatica = [
    'nome', 'codigo', 'setor',
    'ativo_circulante_2020', 'ativo_circulante_2021', 'ativo_circulante_2022', 'ativo_circulante_2023', 'ativo_circulante_2024',
    'ativo_nao_circulante_2020', 'ativo_nao_circulante_2021', 'ativo_nao_circulante_2022', 'ativo_nao_circulante_2023', 'ativo_nao_circulante_2024',
    'passivo_circulante_2020', 'passivo_circulante_2021', 'passivo_circulante_2022', 'passivo_circulante_2023', 'passivo_circulante_2024',
    'passivo_nao_circulante_2020', 'passivo_nao_circulante_2021', 'passivo_nao_circulante_2022', 'passivo_nao_circulante_2023', 'passivo_nao_circulante_2024',
    'patrimonio_liquido_2020', 'patrimonio_liquido_2021', 'patrimonio_liquido_2022', 'patrimonio_liquido_2023', 'patrimonio_liquido_2024',
    'lpa_2020', 'lpa_2021', 'lpa_2022', 'lpa_2023', 'lpa_2024',
    'receita_2020', 'receita_2021', 'receita_2022', 'receita_2023', 'receita_2024',
    'lucro_liquido_2020', 'lucro_liquido_2021', 'lucro_liquido_2022', 'lucro_liquido_2023', 'lucro_liquido_2024',
    'divida_bruta_ativo_2020', 'divida_bruta_ativo_2021', 'divida_bruta_ativo_2022', 'divida_bruta_ativo_2023', 'divida_bruta_ativo_2024',
    'liquidez_corrente_2020', 'liquidez_corrente_2021', 'liquidez_corrente_2022', 'liquidez_corrente_2023', 'liquidez_corrente_2024',
    'liquidez_geral_2020', 'liquidez_geral_2021', 'liquidez_geral_2022', 'liquidez_geral_2023', 'liquidez_geral_2024',
    'rentabilidade_ativo_2020', 'rentabilidade_ativo_2021', 'rentabilidade_ativo_2022', 'rentabilidade_ativo_2023', 'rentabilidade_ativo_2024',
    'roe_2020', 'roe_2021', 'roe_2022', 'roe_2023', 'roe_2024',
    'margem_ebit_2020', 'margem_ebit_2021', 'margem_ebit_2022', 'margem_ebit_2023', 'margem_ebit_2024',
    'margem_liquida_2020', 'margem_liquida_2021', 'margem_liquida_2022', 'margem_liquida_2023', 'margem_liquida_2024',
    'ativo_total_2020', 'ativo_total_2021', 'ativo_total_2022', 'ativo_total_2023', 'ativo_total_2024',
    'caixa_operacional_2020', 'caixa_operacional_2021', 'caixa_operacional_2022', 'caixa_operacional_2023', 'caixa_operacional_2024',
    'caixa_investimento_2020', 'caixa_investimento_2021', 'caixa_investimento_2022', 'caixa_investimento_2023', 'caixa_investimento_2024',
    'caixa_financeiro_2020', 'caixa_financeiro_2021', 'caixa_financeiro_2022', 'caixa_financeiro_2023', 'caixa_financeiro_2024',
    'valor_mercado_2020', 'valor_mercado_2021', 'valor_mercado_2022', 'valor_mercado_2023', 'valor_mercado_2024'
]

df_economatica_renomeado = df_economatica_colunas_removidas.copy()
df_economatica_renomeado.columns = novos_nomes_colunas_df_economatica

df_economatica_renomeado.columns

In [ ]:
novos_nomes_colunas_df_economatica_dividendos = [
    'nome', 'codigo', 'dividendos_2020', 'dividendos_2021', 'dividendos_2022', 'dividendos_2023', 'dividendos_2024'
]

df_economatica_dividendos_renomeado = df_economatica_dividendos_colunas_removidas.copy()
df_economatica_dividendos_renomeado.columns = novos_nomes_colunas_df_economatica_dividendos

df_economatica_dividendos_renomeado.columns

## Verificando integridade dos dados

Antes de criar as novas features, devemos mesclar os dataframes, mas como vimos anteriormente, a quantidade de linhas está diferente, e é isso que iremos analisar neste bloco.

Para fazer isso, basta verificar qual código não está presente no dataset dos dividendos.

In [ ]:
codigos_ausentes_em_economatica_limpo = df_economatica_dividendos_renomeado[
    ~df_economatica_dividendos_renomeado['codigo'].isin(df_economatica_renomeado['codigo'])
]['codigo']

print(f"Foi encontrado {codigos_ausentes_em_economatica_limpo.count()} códigos presentes em df_economatica_dividendos_renomeado, mas ausentes em df_economatica_renomeado:")
codigos_ausentes_em_economatica_limpo

In [ ]:
codigos_ausentes_em_dividendos_limpo = df_economatica_renomeado[
    ~df_economatica_renomeado['codigo'].isin(df_economatica_dividendos_renomeado['codigo'])
]['codigo']

print(f"Foi encontrado {codigos_ausentes_em_dividendos_limpo.count()} códigos presentes em df_economatica_renomeado, mas ausentes em df_economatica_dividendos_renomeado:")
codigos_ausentes_em_dividendos_limpo

In [ ]:
nomes_ausentes_em_economatica_limpo = df_economatica_dividendos_renomeado[
    ~df_economatica_dividendos_renomeado['nome'].isin(df_economatica_renomeado['nome'])
]['nome']

print(f"Foi encontrado {nomes_ausentes_em_economatica_limpo.count()} nomes de empresas presentes em df_economatica_dividendos_renomeado, mas ausentes em df_economatica_renomeado:")
nomes_ausentes_em_economatica_limpo

In [ ]:
nomes_ausentes_em_dividendos_limpo = df_economatica_renomeado[
    ~df_economatica_renomeado['nome'].isin(df_economatica_dividendos_renomeado['nome'])
]['nome']

print(f"\n Foi encontrado {nomes_ausentes_em_dividendos_limpo.count()} nomes de empresas presentes em df_economatica_renomeado, mas ausentes em df_economatica_dividendos_renomeado:")
nomes_ausentes_em_dividendos_limpo

Verificado os valores inconsistentes, iremos removê-los.

In [ ]:
df_economatica_filtrado = df_economatica_renomeado[
    df_economatica_renomeado['codigo'].isin(df_economatica_dividendos_renomeado['codigo'])
].copy()

print(f"Shape de df_economatica_renomeado após remoção: {df_economatica_filtrado.shape}")

In [ ]:
df_economatica_dividendos_filtrado = df_economatica_dividendos_renomeado[
    df_economatica_dividendos_renomeado['codigo'].isin(df_economatica_renomeado['codigo'])
].copy()

print(f"Shape de df_economatica_dividendos_renomeado após remoção: {df_economatica_dividendos_filtrado.shape}")

## Mesclagem dos datasets

Agora que temos os datasets com os mesmos registros, iremos mesclá-los.

In [ ]:
colunas_dividendos_para_mesclar = [col for col in df_economatica_dividendos_filtrado.columns if col not in df_economatica_filtrado.columns or col == 'codigo']

df_economatica_mesclado = pd.merge(
    df_economatica_filtrado,
    df_economatica_dividendos_filtrado[colunas_dividendos_para_mesclar],
    on='codigo',
    how='left'
)

print("Shape do DataFrame mesclado:")
print(df_economatica_mesclado.shape)

In [ ]:
df_economatica_mesclado.info()

In [ ]:
df_economatica_mesclado.columns

Procurando atras do regex '_\d{4}$' todas as colunas que terminam com _ano


In [ ]:
colunas_com_ano = [col for col in df_economatica_mesclado.columns if re.search(r'_\d{4}$', col)]

Dividindo a base das colunas por ano

In [ ]:
base_colunas = defaultdict(list)
for col in colunas_com_ano:
    match = re.match(r"(.*)_(\d{4})$", col)
    if match:
        base, ano = match.groups()
        base_colunas[base].append(col)

Fazendo o melt (extensão dos dados) com o ano

In [ ]:
dfs_long = []
id_vars = [col for col in df_economatica_mesclado.columns if col not in colunas_com_ano]  # colunas que você quer manter (ex: nome, setor, etc.)

for base, cols in base_colunas.items():
    # melt para cada base
    df_long = df_economatica_mesclado.melt(id_vars=id_vars, value_vars=cols,
                      var_name='variavel', value_name=base)
    # extrai o ano
    df_long['ano'] = df_long['variavel'].str.extract(r'(\d{4})$')
    # drop coluna temporária
    df_long = df_long.drop(columns=['variavel'])
    dfs_long.append(df_long)

# Agora você pode juntar tudo pelo id_vars + ano
from functools import reduce

df_economatica_com_ano = reduce(lambda left, right: pd.merge(left, right, on=id_vars + ['ano'], how='outer'), dfs_long)

In [ ]:
df_economatica_com_ano

## Criação de features

### Feature - ano

Agora iremos dividir em 5 datasets, sendo cada um as ações de um ano específico.

In [ ]:
def criar_df_ano(ano: int):
  df_ano = df_economatica_mesclado[['nome', 'codigo', 'setor'] + [col for col in df_economatica_mesclado.columns if col.endswith(f'_{ano}')]].copy()
  df_ano.columns = ['nome', 'codigo', 'setor'] + [col.replace(f'_{ano}', '') for col in df_ano.columns if col.endswith(f'_{ano}')]
  df_ano['ano'] = ano

  print(f"Shape do DataFrame de {ano}:", df_ano.shape)

  return df_ano

In [ ]:
df_economatica_2020 = criar_df_ano(2020)
df_economatica_2021 = criar_df_ano(2021)
df_economatica_2022 = criar_df_ano(2022)
df_economatica_2023 = criar_df_ano(2023)
df_economatica_2024 = criar_df_ano(2024)

In [ ]:
dataframes_anos = [
    df_economatica_2020,
    df_economatica_2021, 
    df_economatica_2022,
    df_economatica_2023,
    df_economatica_2024
]

# Concatenar todos os DataFrames
df_economatica_com_ano = pd.concat(dataframes_anos, ignore_index=True)

print(f"Shape do DataFrame mesclado: {df_economatica_com_ano.shape}")

### Feature - pandemia

Neste trecho iremos adicionar 1 na coluna pandemia nos anos que estavam em alerta emergencial de COVID-19, segundo a OMS, e 0 nos que nao estavam

In [ ]:
df_economatica_manipulado = df_economatica_com_ano.copy()

df_economatica_manipulado['pandemia'] = df_economatica_com_ano['ano'].apply(
    lambda x: 1 if x in [2020, 2021, 2022] else 0
)

print("Coluna 'pandemia' criada com sucesso!")
print(f"Shape do DataFrame: {df_economatica_manipulado.shape}")
print("\nDistribuição da variável pandemia:")
print(df_economatica_manipulado['pandemia'].value_counts())
print("\nDistribuição por ano e pandemia:")
print(df_economatica_manipulado.groupby(['ano', 'pandemia']).size())

### Feature - variacao_valor_mercado (Variável Target)

In [ ]:
df_economatica_manipulado['variacao_valor_mercado'] = np.nan

df_economatica_manipulado['variacao_valor_mercado'].head()

# Limpeza dos dados

Antes e começar a utilizar os dados, precisamos realizar algumas etapas de  limpeza dos dados, como:
- Verificar e tratar dados duplicados.
- Verificar e tratar tipagem.
- Verificar e tratar nulos.

## Tratamento de valores duplicados

In [ ]:
print(f"Número de linhas duplicadas: {df_economatica_manipulado.duplicated().sum()}")

Mesmo que não tenha linhas com todos os valores iguais, irei verificar se não há códigos duplicados.

In [ ]:
print(f"Número de códigos duplicados: {df_economatica_manipulado['codigo'].duplicated().sum()}")

print(f"Número de registros duplicados (mesmo código no mesmo ano): {df_economatica_manipulado.duplicated(subset=['codigo', 'ano']).sum()}")

contagem_por_codigo = df_economatica_manipulado['codigo'].value_counts()
print(f"\nCódigos que aparecem 5 vezes (completos): {(contagem_por_codigo == 5).sum()}")
print(f"Códigos que aparecem menos de 5 vezes (incompletos): {(contagem_por_codigo < 5).sum()}")

Show! Agora sabemos que nenhum registro é duplicado.

## Verificar e tratar tipagem

Vamos iniciar filtrando somente as colunas numéricas.

In [ ]:
colunas_numericas = [col for col in df_economatica_manipulado.columns if col not in ['nome', 'codigo', 'setor']]

### Identificando formato dos valores inconsistentes

In [ ]:
def checagem_de_valores_nao_numericos(df, columns):
    valores_nao_numericos = {}
    for col in columns:
        # Tentar converter a coluna para numérico, com errors='coerce' para transformar valores inválidos em NaN
        # Usamos errors='coerce' aqui apenas para identificar o que NÃO é numérico, sem modificar o DataFrame original
        col_numerica = pd.to_numeric(df[col], errors='coerce')
        # Encontrar os valores no DataFrame original que se tornaram NaN após a coerção e que não eram NaN originalmente
        valores_invalidos = df[col][col_numerica.isnull() & df[col].notnull()]
        if not valores_invalidos.empty:
            valores_nao_numericos[col] = valores_invalidos.unique().tolist()
    return valores_nao_numericos

In [ ]:
def exibir_valores_nao_numericos(df_economatica_mesclado, colunas_numericas):
    valores_nao_numericos_encontrados = checagem_de_valores_nao_numericos(df_economatica_mesclado, colunas_numericas)

    if valores_nao_numericos_encontrados:
        print("Valores não numéricos encontrados nas seguintes colunas:")
        for col, val in valores_nao_numericos_encontrados.items():
            print(f"- {col}: {val}")
    else:
        print("Nenhum valor não numérico encontrado nas colunas a verificar.")

In [ ]:
exibir_valores_nao_numericos(df_economatica_manipulado, colunas_numericas)

Identifiquei padrões:
- Valores nulos representados por: '-'
- Valores com casas decimais representados por: '#,##########'
- Valores com casas de milhar em diante representados por: '#.###.###'

### Tratando a inconsistência dos tipos

In [ ]:
def limpar_e_converter(value, decimais=4):
    """
    Converte um valor dinamicamente identificando se é inteiro, float ou string.
    Trata pontuação brasileira (. para milhares, , para decimais).
    
    Args:
        value: O valor a ser convertido
        decimais: Número de casas decimais para arredondamento (padrão: 4)
        
    Returns:
        int, float ou np.nan dependendo do tipo identificado
    """
    # Se já é numérico, retorna como está
    if isinstance(value, (int, float)):
        if isinstance(value, float):
            return round(value, decimais)
        return value
        
    # Se não é string, tenta converter para string
    if not isinstance(value, str):
        value = str(value)
    
    # Remove espaços e verifica se é valor nulo
    value = value.strip()
    if value == '-' or value == '' or value.lower() in ['nan', 'null', 'none']:
        return np.nan
    
    # Remove pontos (separador de milhares) e substitui vírgula por ponto (decimal)
    # Primeiro identifica o padrão: se tem vírgula, ela é o separador decimal
    if ',' in value:
        # Padrão brasileiro: 1.234.567,89
        # Remove todos os pontos (milhares) e substitui vírgula por ponto (decimal)
        value_clean = value.replace('.', '').replace(',', '.')
    else:
        # Se não tem vírgula, assume padrão americano ou número sem decimais
        # Se tem apenas um ponto e dígitos depois dele <= 3, pode ser decimal
        # Se tem mais de um ponto ou mais de 3 dígitos após o último ponto, são milhares
        parts = value.split('.')
        if len(parts) == 2 and len(parts[1]) <= 3 and len(parts[1]) > 0:
            # Provavelmente decimal: 1234.56
            value_clean = value
        elif len(parts) > 2:
            # Múltiplos pontos: separadores de milhares: 1.234.567
            value_clean = value.replace('.', '')
        else:
            # Um ponto com mais de 3 dígitos ou sem ponto
            value_clean = value.replace('.', '')
    
    # Remove outros caracteres não numéricos (exceto - no início)
    # Preserva sinal negativo no início
    is_negative = value_clean.startswith('-')
    value_clean = re.sub(r'[^\d.]', '', value_clean.lstrip('-'))
    if is_negative:
        value_clean = '-' + value_clean
    
    try:
        # Tenta converter para float primeiro
        float_value = float(value_clean)
        
        # Verifica se é um inteiro (sem parte decimal)
        if float_value.is_integer():
            return int(float_value)
        else:
            return round(float_value, decimais)
            
    except (ValueError, TypeError):
        return np.nan

In [ ]:
def aplicar_conversao(df, decimais=4):
    """
    Aplica a conversão dinâmica para as colunas especificadas.
    
    Args:
        df: DataFrame a ser processado
        decimais: Número de casas decimais para arredondamento (padrão: 4)
        
    Returns:
        DataFrame com as colunas convertidas
    """
    df_convertido = df.copy()
    
    for col in colunas_numericas:
        if col in df_convertido.columns:
            print(f"Convertendo coluna: {col}")
            df_convertido[col] = df_convertido[col].apply(lambda x: limpar_e_converter(x, decimais))
    
    return df_convertido

Antes de tratar, vamos criar uma cópia para podermos comparar o começo e o fim de cada dataframe.

In [ ]:
df_economatica_correcao_tipagem = aplicar_conversao(df_economatica_manipulado)

### Exemplos de uso com diferentes casas decimais

Agora você pode controlar o número de casas decimais passando o parâmetro `decimais`:

In [ ]:
# Exemplo: Aplicar conversão com 2 casas decimais
print("=== Exemplo com 2 casas decimais ===")
df_2_decimais = aplicar_conversao(df_economatica_manipulado, decimais=2)

# Exemplo: Aplicar conversão com 4 casas decimais (padrão)
print("\n=== Exemplo com 4 casas decimais (padrão) ===")
df_4_decimais = aplicar_conversao(df_economatica_manipulado, decimais=4)

# Exemplo: Teste individual da função
print("\n=== Testes individuais da função ===")
print(f"limpar_e_converter('1.234,567', decimais=2) = {limpar_e_converter('1.234,567', decimais=2)}")
print(f"limpar_e_converter('1.234,567', decimais=4) = {limpar_e_converter('1.234,567', decimais=4)}")
print(f"limpar_e_converter('123.45', decimais=2) = {limpar_e_converter('123.45', decimais=2)}")
print(f"limpar_e_converter('123.45', decimais=0) = {limpar_e_converter('123.45', decimais=0)}")

In [ ]:
print("\nTipagem das colunas após tratamento:")
df_economatica_correcao_tipagem.dtypes

In [ ]:
df_economatica_correcao_tipagem.head()

## Tratamento de nulos

In [ ]:
def contar_e_exibir_nulos(df):
    """
    Conta os valores nulos por coluna em um DataFrame e exibe as colunas com nulos.

    Args:
        df (pd.DataFrame): O DataFrame de entrada.
    """
    null_counts = df.isnull().sum()
    columns_with_nulls = null_counts[null_counts > 0]

    print("Contagem de valores nulos por coluna (apenas colunas com nulos):")
    print(columns_with_nulls)
    print(f'\n{columns_with_nulls.sum()} células possuem valores nulos no total.')
    num_linhas_com_nulos = df.isnull().any(axis=1).sum()
    print(f"Número de linhas com pelo menos um valor nulo: {num_linhas_com_nulos}")

In [ ]:
contar_e_exibir_nulos(df_economatica_correcao_tipagem)

São muitas células com valores nulos... Tentaremos adotar as seguintes abordagens:

- criar uma matriz de correlação para eliminar variáveis altamente correlacionadas;
- coletar as colunas derivadas a partir de cálculos de outras colunas e realizar os cálculos (mas para isso precisamos verificar antes se essa coluna utilizada no cálculo possui valor);
- substituir valores nulos com base na média da linha (ex: ativo_circulante de 2020 até de 2023 possuem valores, mas 2024 não, logo coleto a média dos 4 anos para gerar o valor em 2024);
- e por fim, eliminar as ações cujas linhas possuem valores nulos.

### Matriz de Correlação

Nessa etapa iremos selecionar as features que causam a multicolinearidade e remover a que tiver menos correlação com a variável target.

Primeiro selecionamos somente os valores númericos.

Depois criamos uma função para calcular a correlação e exibir um mapa de calor conforme o ano.

In [ ]:
df_numerico = df_economatica_correcao_tipagem[colunas_numericas].astype(float)

In [ ]:
def calcular_e_exibir_correlacao(df: pd.DataFrame):
    """
    Calcula e exibe a matriz de correlação para todas as colunas numéricas.
    
    Args:
        df: DataFrame com os dados
    """
    # Seleciona apenas colunas numéricas (excluindo colunas de identificação)
    colunas_numericas_filtradas = [col for col in df.columns 
                                 if col not in ['nome', 'codigo', 'setor'] 
                                 and df[col].dtype in ['int64', 'float64', 'int32', 'float32']]
    
    if not colunas_numericas_filtradas:
        print("Nenhuma coluna numérica encontrada")
        return
    
    # Seleciona apenas as colunas numéricas
    df_numerico = df[colunas_numericas_filtradas]
    
    # Remove linhas com muitos NaN para melhorar a correlação
    df_numerico_limpo = df_numerico.dropna(thresh=len(colunas_numericas_filtradas)*0.5)
    
    # Calcula a matriz de correlação
    matriz_correlacao = df_numerico_limpo.corr()
    
    # Cria uma máscara para mostrar apenas o triângulo superior
    mask = np.triu(np.ones_like(matriz_correlacao, dtype=bool))
    
    plt.figure(figsize=(20, 18))
    sns.heatmap(matriz_correlacao, 
                mask=mask,
                annot=True, 
                cmap=sns.cubehelix_palette(as_cmap=True),
                fmt='.2f',
                vmin=-1,
                vmax=1,
                linewidths=0.1)
    plt.title(f'Matriz de Correlação')
    plt.tight_layout()
    plt.show()

In [ ]:
def calcular_correlacao_com_receita(df: pd.DataFrame):
    """
    Calcula e exibe a correlação de todas as variáveis numéricas com 'variacao_valor_mercado'.
    """
    # Verifica se a coluna 'variacao_valor_mercado' existe
    if 'variacao_valor_mercado' not in df.columns:
        print("Coluna 'variacao_valor_mercado' não encontrada no DataFrame")
        return
    
    # Seleciona apenas colunas numéricas
    df_numerico = df.select_dtypes(include=[np.number])
    
    # Calcula correlação com a variação do valor de mercado e ordena
    correlacoes = df_numerico.corr()['variacao_valor_mercado'].drop('variacao_valor_mercado').sort_values(key=abs, ascending=False)
    
    # Heatmap vertical - correlação com a variação do valor de mercado
    plt.figure(figsize=(8, 15))
    
    # Cria um DataFrame só com as correlações com a variação do valor de mercado
    correlacoes_df = correlacoes.to_frame(name='Correlação com a variação do valor de mercado')
    
    # Heatmap
    sns.heatmap(correlacoes_df, 
                annot=True, 
                cmap=sns.cubehelix_palette(as_cmap=True),
                center=0,
                fmt='.3f',
                cbar_kws={'label': 'Correlação'},
                linewidths=0.5)
    
    plt.title('Correlação de todas as variáveis com variacao_valor_mercado')
    plt.ylabel('Variáveis')
    plt.tight_layout()
    plt.show()

Antes de sair selecionando as features com alta correlação, vamos definir um threshold (limiar) para considerar se está muito correlacionado ou não, sendo um dos nossos critérios de exclusão.

In [ ]:
calcular_e_exibir_correlacao(df_economatica_correcao_tipagem)

Visualizando de forma geral, parece estar bem dividido entre fortemente correlacionadas ou não. Vamos considerar uma correlação sendo forte como seu valor acima de 0.85.

As seguintes variáveis estão fortemente correlacionadas:
- caixa_financeiro <- **0.91** -> dividendos
- ano <- **-0.87** -> pandemia
- margem_liquida <- **0.96** -> margem_ebit
- receita <- **-0.83** -> dividendos
- lucro_liquido <- **-0.90** -> dividendos
- caixa_financeiro <- **-0.84** -> receita
- caixa_financeiro <- **-0.84** -> lucro_liquido
- caixa_operacional <- **0.81** -> patrimonio_liquido
- caixa_operacional <- **0.90** -> receita
- ativo_total <- **0.91** -> receita
- lucro_liquido <- **0.81** -> patrimonio_liquido
- lucro_liquido <- **0.80** -> receita
- receita <- **0.87** -> patrimonio_liquido

As seguintes variáveis não são consideradas, porque já estão marcadas para remover:
- ativo_circulante
- ativo_nao_circulante
- passivo_circulante
- passivo_nao_circulante

### Colunas derivadas

As colunas derivadas são:

**Endividamento**:
- divida_bruta_ativo_* → já está no dataset

**Liquidez**:
- liquidez_corrente_* = ativo_circulante_* / passivo_circulante_*
- liquidez_geral_* = (ativo_circulante_* + ativo_nao_circulante_*) / (passivo_circulante_* + passivo_nao_circulante_*)

**Rentabilidade**:
- rentabilidade_ativo_* (ROA) = lucro_liquido_* / ((ativo_total_* + ativo_total*_(ano-1)) / 2)
- roe_* = lucro_liquido_* / ((patrimonio_liquido_* + patrimonio_liquido*_(ano-1)) / 2)

**Margens**:
- margem_liquida_* = lucro_liquido_* / receita_*
- margem_ebit_* → já está no dataset (mas não pode ser recalculada, pois falta ebit_*)

**Lucro por Ação**:
- lpa_* → já está no dataset (não dá para recalcular, pois falta numero_acoes_*)

**Valor de Mercado da Empresa**:
- valor_mercado_* → já está no dataset (não dá para recalcular, pois falta preco_acao_31dez_* e numero_acoes_*)


#### Liquidez

In [ ]:
def analisar_nulos_coluna_derivada(df, coluna_derivada, colunas_calculo):
    """
    Analisa valores nulos em uma coluna derivada e verifica valores não nulos nas colunas de cálculo.

    Args:
        df (pd.DataFrame): O DataFrame de entrada.
        coluna_derivada (str): O nome da coluna derivada (ex: 'liquidez_corrente').
        colunas_calculo (list): Uma lista de nomes das colunas usadas para cálculo
                               (ex: ['ativo_circulante', 'passivo_circulante']).
    """
    print(f"Análise de valores nulos para a coluna {coluna_derivada}:")

    # Verifica se a coluna derivada e as colunas de cálculo existem
    if coluna_derivada in df.columns and all(c in df.columns for c in colunas_calculo):
        # Filtra as linhas onde a coluna derivada é nula
        linhas_com_nulos_derivada = df[df[coluna_derivada].isnull()]

        # Constrói a condição para verificar se todas as colunas de cálculo não são nulas nessas linhas
        if not linhas_com_nulos_derivada.empty:
            condicao_nao_nula = linhas_com_nulos_derivada[colunas_calculo[0]].notnull()
            for col_calc in colunas_calculo[1:]:
                condicao_nao_nula = condicao_nao_nula & linhas_com_nulos_derivada[col_calc].notnull()

            # Conta as linhas onde todas as colunas de cálculo não são nulas
            contagem_nao_nula = linhas_com_nulos_derivada[condicao_nao_nula].shape[0]
            
            print(f"Total de linhas com {coluna_derivada} nulo: {len(linhas_com_nulos_derivada)}")
            print(f"Linhas com valores não nulos nas colunas de cálculo: {contagem_nao_nula}")
        else:
            print(f"Nenhuma linha com valor nulo encontrada em {coluna_derivada}")
    else:
        colunas_faltando = [col for col in [coluna_derivada] + colunas_calculo if col not in df.columns]
        print(f"Colunas não encontradas no DataFrame: {colunas_faltando}")

In [ ]:
analisar_nulos_coluna_derivada(df_economatica_correcao_tipagem, 'liquidez_corrente', ['ativo_circulante', 'passivo_circulante'])

In [ ]:
# Aplica a lógica para liquidez_corrente no dataset com ano como feature
# Identifica as linhas onde a coluna derivada é nula e as colunas de cálculo não são nulas
condicao = (df_economatica_correcao_tipagem['liquidez_corrente'].isnull()) & \
           (df_economatica_correcao_tipagem['ativo_circulante'].notnull()) & \
           (df_economatica_correcao_tipagem['passivo_circulante'].notnull()) & \
           (df_economatica_correcao_tipagem['passivo_circulante'] != 0) # Evita divisão por zero

# Calcula o valor derivado
valores_calculados = df_economatica_correcao_tipagem.loc[condicao].apply(
    lambda row: row['ativo_circulante'] / row['passivo_circulante'], axis=1
)

# Aplica a função limpar_e_converter nos valores calculados e preenche os nulos
df_economatica_correcao_tipagem.loc[condicao, 'liquidez_corrente'] = valores_calculados.apply(limpar_e_converter)

print(f"Calculados e preenchidos nulos para 'liquidez_corrente' onde 'ativo_circulante' e 'passivo_circulante' não eram nulos.")

In [ ]:
contar_e_exibir_nulos(df_economatica_correcao_tipagem)

Diminuimos em 10, as células nulas e nenhuma linha ainda...

In [ ]:
analisar_nulos_coluna_derivada(df_economatica_correcao_tipagem, 'liquidez_geral', ['ativo_circulante', 'ativo_nao_circulante', 'passivo_circulante', 'passivo_nao_circulante'])

In [ ]:
condicao = (df_economatica_correcao_tipagem['liquidez_geral'].isnull()) & \
           (df_economatica_correcao_tipagem['ativo_circulante'].notnull()) & \
           (df_economatica_correcao_tipagem['ativo_nao_circulante'].notnull()) & \
           (df_economatica_correcao_tipagem['passivo_circulante'].notnull()) & \
           (df_economatica_correcao_tipagem['passivo_nao_circulante'].notnull())

# Calcula o divisor e evita divisão por zero
divisor = df_economatica_correcao_tipagem.loc[condicao, 'passivo_circulante'] + df_economatica_correcao_tipagem.loc[condicao, 'passivo_nao_circulante']
condicao = condicao & (divisor != 0)

# Calcula o valor derivado
valores_calculados = df_economatica_correcao_tipagem.loc[condicao].apply(
    lambda row: (row['ativo_circulante'] + row['ativo_nao_circulante']) / (row['passivo_circulante'] + row['passivo_nao_circulante']), axis=1
)

# Aplica a função limpar_e_converter nos valores calculados e preenche os nulos
df_economatica_correcao_tipagem.loc[condicao, 'liquidez_geral'] = valores_calculados.apply(limpar_e_converter)

print(f"Calculados e preenchidos nulos para 'liquidez_geral' onde as colunas de cálculo não eram nulas.")

In [ ]:
contar_e_exibir_nulos(df_economatica_correcao_tipagem)

Diminuimos em mais 10, e nenhuma linha recuperada 100% ainda...

#### Margens

In [ ]:
analisar_nulos_coluna_derivada(df_economatica_correcao_tipagem, 'margem_liquida', ['lucro_liquido', 'receita'])

In [ ]:
# Aplica a lógica para margem_liquida no dataset com ano como feature
# Identifica as linhas onde a coluna derivada é nula e as colunas de cálculo não são nulas
condicao = (df_economatica_correcao_tipagem['margem_liquida'].isnull()) & \
           (df_economatica_correcao_tipagem['lucro_liquido'].notnull()) & \
           (df_economatica_correcao_tipagem['receita'].notnull()) & \
           (df_economatica_correcao_tipagem['receita'] != 0) # Evita divisão por zero

# Calcula o valor derivado
valores_calculados = df_economatica_correcao_tipagem.loc[condicao].apply(
    lambda row: row['lucro_liquido'] / row['receita'], axis=1
)

# Aplica a função limpar_e_converter nos valores calculados e preenche os nulos
df_economatica_correcao_tipagem.loc[condicao, 'margem_liquida'] = valores_calculados.apply(limpar_e_converter)

print(f"Calculados e preenchidos nulos para 'margem_liquida' onde 'lucro_liquido' e 'receita' não eram nulos.")

In [ ]:
contar_e_exibir_nulos(df_economatica_correcao_tipagem)

Eliminamos incríveis 1 resultados.

#### Rentabilidade

In [ ]:
analisar_nulos_coluna_derivada(df_economatica_correcao_tipagem, 'rentabilidade_ativo', ['lucro_liquido', 'ativo_total'])

In [ ]:
# Aplica a lógica para rentabilidade_ativo no dataset com ano como feature
# Para calcular ROA, precisamos do ativo total do ano anterior, então criamos um DataFrame auxiliar
df_economatica_sorted = df_economatica_correcao_tipagem.sort_values(['codigo', 'ano']).copy()
df_economatica_sorted['ativo_total_anterior'] = df_economatica_sorted.groupby('codigo')['ativo_total'].shift(1)

# Identifica as linhas onde a coluna derivada é nula e as colunas de cálculo não são nulas
condicao = (df_economatica_sorted['rentabilidade_ativo'].isnull()) & \
           (df_economatica_sorted['lucro_liquido'].notnull()) & \
           (df_economatica_sorted['ativo_total'].notnull()) & \
           (df_economatica_sorted['ativo_total_anterior'].notnull()) & \
           (df_economatica_sorted['ano'] > 2020)  # Só calcula a partir de 2021

# Calcula o denominador (média do ativo total atual e anterior) e evita divisão por zero
denominador = (df_economatica_sorted.loc[condicao, 'ativo_total'] + df_economatica_sorted.loc[condicao, 'ativo_total_anterior']) / 2
condicao = condicao & (denominador != 0)

# Calcula o valor derivado
valores_calculados = df_economatica_sorted.loc[condicao].apply(
    lambda row: row['lucro_liquido'] / ((row['ativo_total'] + row['ativo_total_anterior']) / 2), axis=1
)

# Aplica a função limpar_e_converter nos valores calculados e preenche os nulos
df_economatica_sorted.loc[condicao, 'rentabilidade_ativo'] = valores_calculados.apply(limpar_e_converter)

# Atualiza o DataFrame original
df_economatica_correcao_tipagem = df_economatica_sorted.drop(columns=['ativo_total_anterior']).copy()

print(f"Calculados e preenchidos nulos para 'rentabilidade_ativo' onde as colunas de cálculo não eram nulas.")

In [ ]:
contar_e_exibir_nulos(df_economatica_correcao_tipagem)

In [ ]:
analisar_nulos_coluna_derivada(df_economatica_correcao_tipagem, 'roe', ['lucro_liquido', 'patrimonio_liquido'])

In [ ]:
# Aplica a lógica para roe no dataset com ano como feature
# Para calcular ROE, precisamos do patrimônio líquido do ano anterior
df_economatica_sorted = df_economatica_correcao_tipagem.sort_values(['codigo', 'ano']).copy()
df_economatica_sorted['patrimonio_liquido_anterior'] = df_economatica_sorted.groupby('codigo')['patrimonio_liquido'].shift(1)

# Identifica as linhas onde a coluna derivada é nula e as colunas de cálculo não são nulas
condicao = (df_economatica_sorted['roe'].isnull()) & \
           (df_economatica_sorted['lucro_liquido'].notnull()) & \
           (df_economatica_sorted['patrimonio_liquido'].notnull()) & \
           (df_economatica_sorted['patrimonio_liquido_anterior'].notnull()) & \
           (df_economatica_sorted['ano'] > 2020)  # Só calcula a partir de 2021

# Calcula o denominador (média do patrimônio líquido atual e anterior) e evita divisão por zero
denominador = (df_economatica_sorted.loc[condicao, 'patrimonio_liquido'] + df_economatica_sorted.loc[condicao, 'patrimonio_liquido_anterior']) / 2
condicao = condicao & (denominador != 0)

# Calcula o valor derivado
valores_calculados = df_economatica_sorted.loc[condicao].apply(
    lambda row: row['lucro_liquido'] / ((row['patrimonio_liquido'] + row['patrimonio_liquido_anterior']) / 2), axis=1
)

# Aplica a função limpar_e_converter nos valores calculados e preenche os nulos
df_economatica_sorted.loc[condicao, 'roe'] = valores_calculados.apply(limpar_e_converter)

# Atualiza o DataFrame original
df_economatica_correcao_tipagem = df_economatica_sorted.drop(columns=['patrimonio_liquido_anterior']).copy()

print(f"Calculados e preenchidos nulos para 'roe' onde as colunas de cálculo não eram nulas.")

In [ ]:
contar_e_exibir_nulos(df_economatica_correcao_tipagem)

Aqui já obtivemos resultados significativos, de 648 linhas com valores nulos, fomos para 509.

#### Remover colunas de ativo/passivo 

In [ ]:
df_economatica_correcao_tipagem.drop(columns=['ativo_circulante', 'ativo_nao_circulante', 'passivo_circulante', 'passivo_nao_circulante'], inplace=True)

### Preenchimento com base em média

In [ ]:
def identificar_features_com_nulo_unico_ano(df):
    """
    Identifica features onde há ações que possuem valores nulos em apenas um dos cinco anos (2020-2024).
    
    Args:
        df (pd.DataFrame): DataFrame com dados organizados por ano
        
    Returns:
        dict: Dicionário com features como chave e lista de códigos de ações que atendem o critério
    """
    # Colunas que são features numéricas (excluindo identificadores e categóricas)
    features_numericas = [col for col in df.columns if col not in ['nome', 'codigo', 'setor', 'ano', 'pandemia']]
    
    resultado = {}
    
    for feature in features_numericas:
        acoes_com_nulo_unico = []
        
        # Agrupa por código da ação
        for codigo, grupo in df.groupby('codigo'):
            # Verifica se há exatamente 5 anos de dados (2020-2024)
            if len(grupo) == 5:
                # Conta quantos valores nulos existem para essa feature
                nulos_count = grupo[feature].isnull().sum()
                
                # Se há exatamente 1 valor nulo (nulo em apenas um ano)
                if nulos_count == 1:
                    acoes_com_nulo_unico.append(codigo)
        
        # Se há ações que atendem o critério, adiciona ao resultado
        if acoes_com_nulo_unico:
            resultado[feature] = acoes_com_nulo_unico
    
    return resultado

In [ ]:
def exibir_relatorio_nulos_unicos(df):
    """
    Exibe um relatório detalhado das features com nulos únicos.
    
    Args:
        df (pd.DataFrame): DataFrame com dados organizados por ano
    """
    resultado = identificar_features_com_nulo_unico_ano(df)
    
    if not resultado:
        print("Nenhuma feature possui ações com nulo em apenas um ano.")
        return
    
    print("=== RELATÓRIO: Features com nulos em apenas um ano ===\n")
    
    # Lista apenas os nomes das features
    features_com_nulo_unico = list(resultado.keys())
    print(f"Total de features identificadas: {len(features_com_nulo_unico)}")
    print(f"Features: {features_com_nulo_unico}\n")
    
    # Relatório detalhado por feature
    for feature, acoes in resultado.items():
        print(f"Feature: '{feature}'")
        print(f"  - Quantidade de ações afetadas: {len(acoes)}")
        print(f"  - Códigos das ações: {acoes[:10]}{'...' if len(acoes) > 10 else ''}")
        
        # Mostra em qual ano está o nulo para as primeiras 3 ações
        print("  - Detalhes dos primeiros casos:")
        for i, codigo in enumerate(acoes[:3]):
            dados_acao = df[df['codigo'] == codigo][['ano', feature]]
            ano_nulo = dados_acao[dados_acao[feature].isnull()]['ano'].iloc[0]
            print(f"    * {codigo}: nulo no ano {ano_nulo}")
        print()
    
    return features_com_nulo_unico

In [ ]:
features_nulo_unico = exibir_relatorio_nulos_unicos(df_economatica_correcao_tipagem)

In [ ]:
def preencher_nulos_com_media_temporal(df):
    """
    Preenche valores nulos com a média dos outros anos para ações que possuem 
    exatamente 1 valor nulo em uma feature (4 valores válidos e 1 nulo).
    
    Args:
        df (pd.DataFrame): DataFrame com dados organizados por ano
        
    Returns:
        pd.DataFrame: DataFrame com os valores nulos preenchidos
        dict: Relatório das alterações realizadas
    """
    df_resultado = df.copy()
    relatorio = {
        'features_processadas': [],
        'total_valores_preenchidos': 0,
        'detalhes_por_feature': {}
    }
    
    # Identifica features com nulos únicos
    features_com_nulo_unico = identificar_features_com_nulo_unico_ano(df)
    
    if not features_com_nulo_unico:
        print("Nenhuma feature encontrada com o critério de 1 nulo em 5 anos.")
        return df_resultado, relatorio
    
    print("=== PREENCHIMENTO COM MÉDIA TEMPORAL ===\n")
    
    for feature, acoes_afetadas in features_com_nulo_unico.items():
        valores_preenchidos = 0
        detalhes_acoes = []
        
        print(f"Processando feature: '{feature}'")
        print(f"  - Ações afetadas: {len(acoes_afetadas)}")
        
        for codigo in acoes_afetadas:
            # Obtém os dados da ação para todos os anos
            dados_acao = df_resultado[df_resultado['codigo'] == codigo].copy()
            
            if len(dados_acao) == 5:  # Confirma que tem 5 anos de dados
                valores_feature = dados_acao[feature]
                
                # Verifica se há exatamente 1 nulo
                if valores_feature.isnull().sum() == 1:
                    # Calcula a média dos valores não nulos
                    valores_nao_nulos = valores_feature.dropna()
                    
                    if len(valores_nao_nulos) == 4:  # Confirma que tem 4 valores válidos
                        media = valores_nao_nulos.mean()
                        
                        # Identifica o índice onde está o nulo
                        indice_nulo = dados_acao[valores_feature.isnull()].index[0]
                        ano_nulo = dados_acao.loc[indice_nulo, 'ano']
                        
                        # Aplica a função limpar_e_converter para garantir formatação
                        media_formatada = limpar_e_converter(media)
                        
                        # Preenche o valor nulo com a média
                        df_resultado.loc[indice_nulo, feature] = media_formatada
                        
                        valores_preenchidos += 1
                        detalhes_acoes.append({
                            'codigo': codigo,
                            'ano_nulo': ano_nulo,
                            'media_calculada': media_formatada,
                            'valores_originais': valores_nao_nulos.tolist()
                        })
        
        # Atualiza o relatório
        relatorio['features_processadas'].append(feature)
        relatorio['total_valores_preenchidos'] += valores_preenchidos
        relatorio['detalhes_por_feature'][feature] = {
            'valores_preenchidos': valores_preenchidos,
            'acoes_processadas': detalhes_acoes[:5]  # Mostra apenas os primeiros 5 exemplos
        }
        
        print(f"  - Valores preenchidos: {valores_preenchidos}")
        
        # Mostra alguns exemplos
        if detalhes_acoes:
            print("  - Exemplos:")
            for exemplo in detalhes_acoes[:3]:
                print(f"    * {exemplo['codigo']} (ano {exemplo['ano_nulo']}): "
                      f"média = {exemplo['media_calculada']}")
        print()
    
    print(f"=== RESUMO ===")
    print(f"Features processadas: {len(relatorio['features_processadas'])}")
    print(f"Total de valores preenchidos: {relatorio['total_valores_preenchidos']}")
    
    return df_resultado, relatorio

In [ ]:
def exibir_relatorio_preenchimento(relatorio):
    """
    Exibe um relatório detalhado do preenchimento realizado.
    
    Args:
        relatorio (dict): Relatório retornado pela função de preenchimento
    """
    print("=== RELATÓRIO DETALHADO DE PREENCHIMENTO ===\n")
    
    if not relatorio['features_processadas']:
        print("Nenhum preenchimento foi realizado.")
        return
    
    for feature in relatorio['features_processadas']:
        detalhes = relatorio['detalhes_por_feature'][feature]
        print(f"Feature: '{feature}'")
        print(f"  - Valores preenchidos: {detalhes['valores_preenchidos']}")
        
        if detalhes['acoes_processadas']:
            print("  - Detalhes das ações processadas:")
            for acao in detalhes['acoes_processadas']:
                print(f"    * {acao['codigo']} (ano {acao['ano_nulo']}): "
                      f"média = {acao['media_calculada']}")
                print(f"      Valores originais: {acao['valores_originais']}")
        print()

In [ ]:
print("Antes do preenchimento:")
contar_e_exibir_nulos(df_economatica_correcao_tipagem)
print("\n" + "="*50 + "\n")

df_economatica_preenchido, relatorio_preenchimento = preencher_nulos_com_media_temporal(df_economatica_correcao_tipagem)

print("\n" + "="*50 + "\n")
print("Após o preenchimento:")
contar_e_exibir_nulos(df_economatica_preenchido)

### Atribuição de valor para a variável target

In [ ]:
# Primeiro, garantir que a coluna valor_mercado esteja corretamente limpa e convertida
df_economatica_preenchido['valor_mercado'] = df_economatica_preenchido['valor_mercado'].apply(limpar_e_converter)

# Ordenar por código e ano para cálculo correto
df_economatica_preenchido = df_economatica_preenchido.sort_values(by=['codigo', 'ano'])

# Calcular valor de mercado do ano anterior
df_economatica_preenchido['valor_mercado_anterior'] = df_economatica_preenchido.groupby('codigo')['valor_mercado'].shift(1)

# Calcular a variação do valor de mercado
df_economatica_preenchido['variacao_valor_mercado_raw'] = (df_economatica_preenchido['valor_mercado'] / df_economatica_preenchido['valor_mercado_anterior']) - 1

# Aplicar a lógica da variável target:
# - Se valor_mercado for NaN, target = NaN
# - Se não há ano anterior (primeiro ano), target = NaN
# - Caso contrário, retorna a variação arredondada em 4 casas decimais
def calcular_target(row):
    """
    Calcula a variável target baseada nas regras definidas:
    - Se valor_mercado é NaN, retorna NaN
    - Se não há ano anterior, retorna NaN
    - Caso contrário, retorna a variação arredondada em 4 casas decimais
    """
    if pd.isna(row['valor_mercado']) or pd.isna(row['valor_mercado_anterior']):
        return np.nan
    
    variacao = row['variacao_valor_mercado_raw']
    
    if pd.isna(variacao):
        return np.nan
    else:
        return round(variacao, 4)

# Aplicar a função para calcular a target
df_economatica_preenchido['variacao_valor_mercado'] = df_economatica_preenchido.apply(calcular_target, axis=1)

# Remover colunas auxiliares
df_economatica_preenchido = df_economatica_preenchido.drop(columns=['valor_mercado', 'valor_mercado_anterior', 'variacao_valor_mercado_raw'])

print("--- Variável Target criada ---")
print("Target: Variação percentual do valor de mercado arredondada em 4 casas decimais")
print("Valores positivos = valorização, valores negativos = desvalorização")
print("Valores NaN = primeiro ano de cada empresa (não há ano anterior para comparação)\n")

print("Estatísticas da variável target:")
print(df_economatica_preenchido['variacao_valor_mercado'].describe())

print(f"\nValores nulos na target: {df_economatica_preenchido['variacao_valor_mercado'].isnull().sum()}")
print(f"Tipo da variável target (valores não nulos): {df_economatica_preenchido['variacao_valor_mercado'].dropna().dtype}")

# Calcula estatísticas apenas dos valores não nulos
valores_nao_nulos = df_economatica_preenchido['variacao_valor_mercado'].dropna()
if len(valores_nao_nulos) > 0:
    valorizacoes = (valores_nao_nulos > 0).sum()
    total_validos = len(valores_nao_nulos)
    print(f"Percentual de valorizações (variação > 0): {(valorizacoes / total_validos) * 100:.1f}%")
    print(f"Variação média: {valores_nao_nulos.mean():.4f}")
    print(f"Variação mínima: {valores_nao_nulos.min():.4f}")
    print(f"Variação máxima: {valores_nao_nulos.max():.4f}")

print("\nVisualização do DataFrame final:")
print(df_economatica_preenchido[['nome', 'codigo', 'ano', 'variacao_valor_mercado']].head(20))

print(f"\nDistribuição por ano:")
print(df_economatica_preenchido.groupby('ano')['variacao_valor_mercado'].agg(['count', lambda x: x.isnull().sum()]).rename(columns={'<lambda>': 'nulos'}))

In [ ]:
print("\n📅 DISTRIBUIÇÃO POR ANO:")
distribuicao_ano = df_economatica_preenchido['ano'].value_counts().sort_index()
for ano, count in distribuicao_ano.items():
  print(f"  - {ano}: {count:,} registros")

### Remoção através da correlação

In [ ]:
calcular_correlacao_com_receita(df_economatica_preenchido)

### Análise de Correlação e Remoção de Variáveis

Com base nos dados de correlação fornecidos, a principal estratégia é identificar e remover as variáveis que mostram uma forte multicolinearidade (alta correlação entre si) e que, ao mesmo tempo, têm a menor correlação com a variável-alvo, que é **variacao_valor_mercado**.

---

#### Frequência de Ocorrência em Pares Correlacionados

Primeiro, foi analisada a frequência com que cada variável aparece em um par de alta correlação:

- **receita:** 5 ocorrências  
- **lucro_liquido:** 4 ocorrências  
- **caixa_financeiro:** 3 ocorrências  
- **dividendos:** 3 ocorrências  
- **patrimonio_liquido:** 3 ocorrências  
- **caixa_operacional:** 2 ocorrências  
- **margem_liquida:** 1 ocorrência  
- **margem_ebit:** 1 ocorrência  
- **pandemia:** 1 ocorrência  
- **ano:** 1 ocorrência  
- **ativo_total:** 1 ocorrência  

---

#### Decisão de Remoção por Par

Em cada par de variáveis com forte correlação, a decisão de qual remover foi baseada na que possui a correlação mais próxima de zero com a variável **variacao_valor_mercado**:

- **caixa_financeiro vs. dividendos:**  
  - caixa_financeiro (-0.007) < dividendos (0.008) → **Remover caixa_financeiro**  

- **ano vs. pandemia:**  
  - ano (-0.016) > pandemia (-0.133) → **Remover ano**  

- **margem_liquida vs. margem_ebit:**  
  - margem_liquida (-0.012) > margem_ebit (-0.014) → **Remover margem_liquida**  

- **receita vs. dividendos:**  
  - receita (-0.008) < dividendos (0.008) → **Remover receita**  

- **lucro_liquido vs. dividendos:**  
  - lucro_liquido (0.020) > dividendos (0.008) → **Remover dividendos**  

- **caixa_financeiro vs. receita:**  
  - caixa_financeiro (-0.007) > receita (-0.008) → **Remover caixa_financeiro**  

- **caixa_financeiro vs. lucro_liquido:**  
  - caixa_financeiro (-0.007) < lucro_liquido (0.020) → **Remover caixa_financeiro**  

- **caixa_operacional vs. patrimonio_liquido:**  
  - patrimonio_liquido (0.006) < caixa_operacional (0.023) → **Remover patrimonio_liquido**  

- **caixa_operacional vs. receita:**  
  - receita (-0.008) < caixa_operacional (0.023) → **Remover receita**  

- **ativo_total vs. receita:**  
  - ativo_total (0.007) < receita (-0.008) → **Remover ativo_total**  

- **lucro_liquido vs. patrimonio_liquido:**  
  - patrimonio_liquido (0.006) < lucro_liquido (0.020) → **Remover patrimonio_liquido**  

- **lucro_liquido vs. receita:**  
  - receita (-0.008) < lucro_liquido (0.020) → **Remover receita**  

- **receita vs. patrimonio_liquido:**  
  - patrimonio_liquido (0.006) < receita (-0.008) → **Remover patrimonio_liquido**  

---

#### Lista Final de Variáveis para Remoção

Com base na análise, estas são as variáveis que, consistentemente, devem ser removidas para otimizar o modelo:

- **receita** (5 indicações de remoção)  
- **caixa_financeiro** (3 indicações de remoção)  
- **patrimonio_liquido** (3 indicações de remoção)  
- **dividendos** (2 indicações de remoção)  
- **ano** (1 indicação de remoção)  
- **margem_liquida** (1 indicação de remoção)  
- **ativo_total** (1 indicação de remoção)  


In [ ]:
df_economatica_preenchido.drop(columns=['receita', 'caixa_financeiro', 'patrimonio_liquido', 'dividendos', 'margem_liquida', 'ativo_total'], inplace=True)

In [ ]:
print("\n📅 DISTRIBUIÇÃO POR ANO:")
distribuicao_ano = df_economatica_preenchido['ano'].value_counts().sort_index()
for ano, count in distribuicao_ano.items():
    print(f"  - {ano}: {count:,} registros")

In [ ]:
contar_e_exibir_nulos(df_economatica_preenchido)

### Remover o ano de 2020

In [ ]:
# Remover todos os registros do ano 2020
df_economatica_preenchido = df_economatica_preenchido[df_economatica_preenchido['ano'] != 2020].copy()

print("=== REMOÇÃO DOS REGISTROS DE 2020 ===")
print(f"Registros restantes após remoção do ano 2020:")
print(f"Shape do DataFrame: {df_economatica_preenchido.shape}")

print("\nDistribuição por ano após remoção:")
print(df_economatica_preenchido['ano'].value_counts().sort_index())

print("\nDistribuição da pandemia após remoção:")
print(df_economatica_preenchido['pandemia'].value_counts())

print("\nVerificação - anos únicos restantes:")
print(sorted(df_economatica_preenchido['ano'].unique()))

print("\nDistribuição da variável target após remoção de 2020:")
print(df_economatica_preenchido['variacao_valor_mercado'].value_counts().sort_index())

print(f"\nValores nulos na target após remoção: {df_economatica_preenchido['variacao_valor_mercado'].isnull().sum()}")

# Verifica quantos códigos únicos restaram
print(f"\nCódigos únicos restantes: {df_economatica_preenchido['codigo'].nunique()}")

# Verifica se algum código ficou com menos de 4 anos de dados
contagem_por_codigo = df_economatica_preenchido['codigo'].value_counts()
codigos_incompletos = contagem_por_codigo[contagem_por_codigo < 4]
print(f"Códigos com menos de 4 anos de dados: {len(codigos_incompletos)}")

if len(codigos_incompletos) > 0:
    print("Códigos incompletos:", codigos_incompletos.head())

### Remoção dos nulos restantes

In [ ]:
def identificar_codigos_com_nulos(df):
    """
    Identifica as linhas que possuem valores nulos e retorna a lista de códigos únicos.
    
    Args:
        df (pd.DataFrame): DataFrame com dados organizados por ano
        
    Returns:
        dict: Dicionário com informações detalhadas sobre códigos com nulos
    """
    # Identifica linhas que possuem pelo menos um valor nulo
    linhas_com_nulos = df.isnull().any(axis=1)
    
    # Filtra o DataFrame para apenas linhas com nulos
    df_com_nulos = df[linhas_com_nulos].copy()
    
    # Lista de códigos únicos que possuem linhas com nulos
    codigos_com_nulos = df_com_nulos['codigo'].unique().tolist()
    
    # Contagem de linhas com nulos por código
    contagem_por_codigo = df_com_nulos['codigo'].value_counts().to_dict()
    
    # Análise detalhada por código
    detalhes_por_codigo = {}
    for codigo in codigos_com_nulos:
        dados_codigo = df[df['codigo'] == codigo]
        linhas_nulas_codigo = dados_codigo.isnull().any(axis=1).sum()
        total_linhas_codigo = len(dados_codigo)
        anos_com_nulos = dados_codigo[dados_codigo.isnull().any(axis=1)]['ano'].tolist()
        
        detalhes_por_codigo[codigo] = {
            'total_linhas': total_linhas_codigo,
            'linhas_com_nulos': linhas_nulas_codigo,
            'anos_com_nulos': anos_com_nulos,
            'percentual_nulos': round((linhas_nulas_codigo / total_linhas_codigo) * 100, 2)
        }
    
    # Resumo geral
    resumo = {
        'total_linhas_dataset': len(df),
        'total_linhas_com_nulos': len(df_com_nulos),
        'total_codigos_unicos': df['codigo'].nunique(),
        'total_codigos_com_nulos': len(codigos_com_nulos),
        'percentual_codigos_afetados': round((len(codigos_com_nulos) / df['codigo'].nunique()) * 100, 2)
    }
    
    return {
        'codigos_com_nulos': codigos_com_nulos,
        'contagem_por_codigo': contagem_por_codigo,
        'detalhes_por_codigo': detalhes_por_codigo,
        'resumo': resumo
    }

In [ ]:
def exibir_relatorio_codigos_nulos(df):
    """
    Exibe um relatório completo dos códigos que possuem linhas com valores nulos.
    
    Args:
        df (pd.DataFrame): DataFrame com dados organizados por ano
    """
    resultado = identificar_codigos_com_nulos(df)
    
    print("=== RELATÓRIO: Códigos com Valores Nulos ===\n")
    
    # Resumo geral
    resumo = resultado['resumo']
    print(f"📊 RESUMO GERAL:")
    print(f"  - Total de linhas no dataset: {resumo['total_linhas_dataset']:,}")
    print(f"  - Linhas com pelo menos um nulo: {resumo['total_linhas_com_nulos']:,}")
    print(f"  - Total de códigos únicos: {resumo['total_codigos_unicos']:,}")
    print(f"  - Códigos com nulos: {resumo['total_codigos_com_nulos']:,}")
    print(f"  - Percentual de códigos afetados: {resumo['percentual_codigos_afetados']}%")
    print()
    
    # Lista dos códigos
    codigos = resultado['codigos_com_nulos']
    print(f"📋 LISTA DE CÓDIGOS COM NULOS ({len(codigos)} códigos):")
    
    # Mostra em grupos de 10 para melhor visualização
    for i in range(0, len(codigos), 10):
        grupo = codigos[i:i+10]
        print(f"  {grupo}")
    print()
    
    # Top 10 códigos com mais linhas nulas
    contagem = resultado['contagem_por_codigo']
    top_10_nulos = sorted(contagem.items(), key=lambda x: x[1], reverse=True)[:10]
    
    print(f"🔝 TOP 10 CÓDIGOS COM MAIS LINHAS NULAS:")
    for codigo, qtd_linhas in top_10_nulos:
        detalhes = resultado['detalhes_por_codigo'][codigo]
        print(f"  - {codigo}: {qtd_linhas}/{detalhes['total_linhas']} linhas "
              f"({detalhes['percentual_nulos']}%) - Anos: {detalhes['anos_com_nulos']}")
    print()
    
    return resultado

In [ ]:
def obter_lista_codigos_com_nulos(df):
    """
    Retorna apenas a lista simples de códigos únicos que possuem valores nulos.
    
    Args:
        df (pd.DataFrame): DataFrame com dados organizados por ano
        
    Returns:
        list: Lista de códigos únicos com valores nulos
    """
    resultado = identificar_codigos_com_nulos(df)
    return resultado['codigos_com_nulos']

In [ ]:
def analisar_padroes_nulos_por_codigo(df, codigo_especifico=None):
    """
    Analisa padrões detalhados de nulos para um código específico ou todos os códigos.
    
    Args:
        df (pd.DataFrame): DataFrame com dados organizados por ano
        codigo_especifico (str, optional): Código específico para análise detalhada
        
    Returns:
        dict: Análise detalhada dos padrões de nulos
    """
    if codigo_especifico:
        # Análise para um código específico
        dados_codigo = df[df['codigo'] == codigo_especifico].copy()
        if dados_codigo.empty:
            return f"Código '{codigo_especifico}' não encontrado no dataset."
        
        print(f"=== ANÁLISE DETALHADA: {codigo_especifico} ===\n")
        
        # Para cada ano, mostra quais colunas têm nulos
        for _, linha in dados_codigo.iterrows():
            ano = linha['ano']
            colunas_nulas = linha.isnull()
            colunas_com_nulos = colunas_nulas[colunas_nulas].index.tolist()
            
            print(f"📅 Ano {ano}:")
            if colunas_com_nulos:
                # Remove colunas de identificação da lista
                colunas_features_nulas = [col for col in colunas_com_nulos 
                                        if col not in ['nome', 'codigo', 'setor']]
                print(f"  - Colunas com nulos: {colunas_features_nulas}")
                print(f"  - Total de nulos: {len(colunas_features_nulas)}")
            else:
                print("  - Sem valores nulos")
            print()
    else:
        # Análise geral de padrões
        resultado = identificar_codigos_com_nulos(df)
        return resultado

In [ ]:
print("=== IDENTIFICAÇÃO DE CÓDIGOS COM VALORES NULOS ===\n")
relatorio_nulos = exibir_relatorio_codigos_nulos(df_economatica_preenchido)

In [ ]:
def remover_acoes_com_nulos(df):
    """
    Remove TODAS as ações que possuem pelo menos 1 valor nulo em qualquer ano/feature.
    
    Args:
        df (pd.DataFrame): DataFrame com dados organizados por ano
        
    Returns:
        pd.DataFrame: DataFrame apenas com ações que não possuem nenhum valor nulo
        dict: Relatório das remoções realizadas
    """
    print("=== REMOÇÃO DE AÇÕES COM VALORES NULOS ===\n")
    
    # Estado inicial
    total_linhas_inicial = len(df)
    total_codigos_inicial = df['codigo'].nunique()
    
    print(f"📊 ESTADO INICIAL:")
    print(f"  - Total de linhas: {total_linhas_inicial:,}")
    print(f"  - Total de códigos únicos: {total_codigos_inicial:,}")
    print()
    
    # Identifica códigos com nulos
    resultado_nulos = identificar_codigos_com_nulos(df)
    codigos_com_nulos = resultado_nulos['codigos_com_nulos']
    
    print(f"🔍 ANÁLISE DE NULOS:")
    print(f"  - Códigos com pelo menos 1 nulo: {len(codigos_com_nulos):,}")
    print(f"  - Códigos sem nulos: {total_codigos_inicial - len(codigos_com_nulos):,}")
    print()
    
    # Remove todas as linhas dos códigos que possuem nulos
    df_limpo = df[~df['codigo'].isin(codigos_com_nulos)].copy()
    
    # Estado final
    total_linhas_final = len(df_limpo)
    total_codigos_final = df_limpo['codigo'].nunique()
    
    linhas_removidas = total_linhas_inicial - total_linhas_final
    codigos_removidos = total_codigos_inicial - total_codigos_final
    
    print(f"🗑️ REMOÇÕES REALIZADAS:")
    print(f"  - Linhas removidas: {linhas_removidas:,}")
    print(f"  - Códigos removidos: {codigos_removidos:,}")
    print()
    
    print(f"✅ ESTADO FINAL:")
    print(f"  - Total de linhas: {total_linhas_final:,}")
    print(f"  - Total de códigos únicos: {total_codigos_final:,}")
    print(f"  - Percentual de linhas mantidas: {(total_linhas_final/total_linhas_inicial)*100:.1f}%")
    print(f"  - Percentual de códigos mantidos: {(total_codigos_final/total_codigos_inicial)*100:.1f}%")
    print()
    
    # Verifica se ainda há nulos (deve ser 0)
    nulos_restantes = df_limpo.isnull().sum().sum()
    print(f"🔍 VERIFICAÇÃO FINAL:")
    print(f"  - Valores nulos restantes: {nulos_restantes}")
    
    if nulos_restantes == 0:
        print("  ✅ Sucesso! Nenhum valor nulo encontrado no dataset final.")
    else:
        print("  ⚠️ Atenção! Ainda há valores nulos no dataset.")
    
    # Relatório detalhado
    relatorio = {
        'estado_inicial': {
            'total_linhas': total_linhas_inicial,
            'total_codigos': total_codigos_inicial
        },
        'estado_final': {
            'total_linhas': total_linhas_final,
            'total_codigos': total_codigos_final
        },
        'remocoes': {
            'linhas_removidas': linhas_removidas,
            'codigos_removidos': codigos_removidos,
            'codigos_com_nulos': codigos_com_nulos
        },
        'percentuais': {
            'linhas_mantidas': round((total_linhas_final/total_linhas_inicial)*100, 2),
            'codigos_mantidos': round((total_codigos_final/total_codigos_inicial)*100, 2)
        },
        'verificacao': {
            'nulos_restantes': nulos_restantes,
            'dataset_limpo': nulos_restantes == 0
        }
    }
    
    return df_limpo, relatorio

In [ ]:
def exibir_estatisticas_pos_remocao(df_limpo, relatorio):
    """
    Exibe estatísticas detalhadas após a remoção das ações com nulos.
    
    Args:
        df_limpo (pd.DataFrame): DataFrame após remoção
        relatorio (dict): Relatório da remoção
    """
    print("\n" + "="*60)
    print("📈 ESTATÍSTICAS DETALHADAS PÓS-REMOÇÃO")
    print("="*60)
    
    # Distribuição por ano
    print("\n📅 DISTRIBUIÇÃO POR ANO:")
    distribuicao_ano = df_limpo['ano'].value_counts().sort_index()
    for ano, count in distribuicao_ano.items():
        print(f"  - {ano}: {count:,} registros")
    
    # Verifica se todos os códigos têm 4 anos de dados
    contagem_por_codigo = df_limpo['codigo'].value_counts()
    codigos_completos = (contagem_por_codigo == 4).sum()
    codigos_incompletos = (contagem_por_codigo != 4).sum()
    
    print(f"\n🔍 INTEGRIDADE DOS DADOS:")
    print(f"  - Códigos com 4 anos completos: {codigos_completos:,}")
    print(f"  - Códigos com dados incompletos: {codigos_incompletos:,}")
    
    if codigos_incompletos > 0:
        print(f"  ⚠️ Atenção: {codigos_incompletos} códigos não têm 4 anos de dados!")
        incompletos = contagem_por_codigo[contagem_por_codigo != 4]
        print(f"  - Detalhes: {incompletos.to_dict()}")
    
    # Estatísticas de colunas
    print(f"\n📊 ESTATÍSTICAS DO DATASET:")
    print(f"  - Número de colunas: {df_limpo.shape[1]}")
    print(f"  - Colunas numéricas: {len(df_limpo.select_dtypes(include=[np.number]).columns)}")
    print(f"  - Densidade dos dados: 100% (sem nulos)")
    
    # Lista algumas ações que permaneceram
    print(f"\n✅ EXEMPLOS DE AÇÕES MANTIDAS:")
    acoes_exemplo = df_limpo['codigo'].unique()[:10]
    print(f"  - Primeiras 10: {list(acoes_exemplo)}")
    
    return distribuicao_ano, contagem_por_codigo

In [ ]:
print("ANTES DA REMOÇÃO:")
contar_e_exibir_nulos(df_economatica_preenchido)
print("\n" + "="*60 + "\n")

df_economatica_sem_nulos, relatorio_remocao = remover_acoes_com_nulos(df_economatica_preenchido)

# Exibe estatísticas detalhadas
estatisticas = exibir_estatisticas_pos_remocao(df_economatica_sem_nulos, relatorio_remocao)

print("\n" + "="*60)
print("🎯 VERIFICAÇÃO FINAL DE NULOS:")
contar_e_exibir_nulos(df_economatica_sem_nulos)

In [ ]:
print("\n📅 DISTRIBUIÇÃO POR ANO:")
distribuicao_ano = df_economatica_sem_nulos['ano'].value_counts().sort_index()
for ano, count in distribuicao_ano.items():
  print(f"  - {ano}: {count:,} registros")

## Gerar CSV do dataframe limpo

In [ ]:
# Gerar CSV do dataframe limpo
print("=== EXPORTAÇÃO DO DATASET LIMPO ===\n")

# Define o caminho baseado no ambiente
if is_colab:
    caminho_csv_limpo = '/content/df_economatica_limpo.csv'
else:
    os.makedirs('../datasets', exist_ok=True)
    caminho_csv_limpo = '../datasets/df_economatica_limpo.csv'

# Exporta o CSV
df_economatica_sem_nulos.to_csv(caminho_csv_limpo, index=False, encoding='utf-8')

print(f"✅ Dataset limpo exportado para: {caminho_csv_limpo}")
print(f"📊 Shape do dataset: {df_economatica_sem_nulos.shape}")
print(f"📅 Período: {df_economatica_sem_nulos['ano'].min()} - {df_economatica_sem_nulos['ano'].max()}")
print(f"🏢 Empresas únicas: {df_economatica_sem_nulos['codigo'].nunique()}")
print(f"📈 Colunas: {df_economatica_sem_nulos.shape[1]}")

# Resumo das colunas
print(f"\n📋 RESUMO DAS COLUNAS:")
colunas_por_tipo = {
    'Identificação': ['nome', 'codigo'],
    'Categóricas': ['setor'],
    'Temporais': ['ano', 'pandemia'],
    'Target': ['variacao_valor_mercado'],
    'Features Financeiras': [col for col in df_economatica_sem_nulos.columns 
                           if col not in ['nome', 'codigo', 'setor', 'ano', 'pandemia', 'variacao_valor_mercado']]
}

for tipo, colunas in colunas_por_tipo.items():
    colunas_existentes = [col for col in colunas if col in df_economatica_sem_nulos.columns]
    print(f"  - {tipo}: {len(colunas_existentes)} colunas")

print(f"\n🔍 VERIFICAÇÕES FINAIS:")
print(f"  - Valores nulos: {df_economatica_sem_nulos.isnull().sum().sum()}")
print(f"  - Linhas duplicadas: {df_economatica_sem_nulos.duplicated().sum()}")
print(f"  - Registros por empresa: {df_economatica_sem_nulos['codigo'].value_counts().unique()}")

# Mostra amostra dos dados
print(f"\n👀 AMOSTRA DOS DADOS:")
print(df_economatica_sem_nulos.head())

print(f"\n📁 Arquivo salvo com sucesso!")
print(f"   Caminho: {caminho_csv_limpo}")
print(f"   Tamanho estimado: ~{df_economatica_sem_nulos.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Análise exploratória

In [ ]:
df_economatica_eda = df_economatica_sem_nulos.copy()
df_economatica_eda.reset_index(drop=True, inplace=True)

## Estatística descritiva

Vamos tentar identificar grande esparcidade nos dados.

In [ ]:
df_economatica_eda.describe()

In [ ]:
# Iterando sobre todas as colunas numéricas e mostrando min, max e média
print("=== ESTATÍSTICAS DAS COLUNAS NUMÉRICAS ===\n")

colunas_numericas = [col for col in df_economatica_eda.columns if col not in ['nome', 'codigo', 'setor', 'pandemia', 'ano']]

print(f"Total de colunas numéricas encontradas: {len(colunas_numericas)}\n")
print("-" * 80)

for i, coluna in enumerate(colunas_numericas, 1):
    print(f"{i:2d}. Coluna: {coluna}")
    print(f"    Mínimo: {df_economatica_eda[coluna].min():.4f}")
    print(f"    Máximo: {df_economatica_eda[coluna].max():.4f}")
    print(f"    Média:  {df_economatica_eda[coluna].mean():.4f}")
    print(f"    Amplitude total: {(df_economatica_eda[coluna].max() - df_economatica_eda[coluna].min()):.4f}")
    print(f"    Variância: {df_economatica_eda[coluna].var(ddof=1):.4f}")
    print(f"    Desvio padrão: {df_economatica_eda[coluna].std(ddof=0):.4f}")
    print(f"    Assimetria:  {df_economatica_eda[coluna].skew():.4f}")
    print(f"    Curtose:  {df_economatica_eda[coluna].kurtosis():.4f}")
    print("-" * 40)

Sobre cada variável:
- **Mínimo:** Representa o menor valor dessa coluna no dataframe.
- **Máximo:** Representa o maior valor dessa coluna no dataframe.
- **Média:** Representa a média aritmética de todos os valores da coluna.
- **Amplitude total:** Representa a diferença entre o valor máximo e mínimo da coluna (max - min), indicando a dispersão dos dados.
- **Variância:** Mede o grau de dispersão dos dados em relação à média. Valores altos indicam maior variabilidade.
- **Desvio Padrão:** Representa a raiz quadrada da variância, medindo a dispersão dos dados em relação à média na mesma unidade dos dados originais. Quanto maior o desvio padrão, mais dispersos estão os dados.
- **Assimetria (Skewness):** Mede a assimetria da distribuição dos dados:
  - Valor = 0: distribuição simétrica
  - Valor > 0: assimetria positiva (cauda à direita)
  - Valor < 0: assimetria negativa (cauda à esquerda)
- **Curtose (Kurtosis):** Mede o "achatamento" da distribuição:
  - Valor = 0: distribuição normal (mesocúrtica)
  - Valor > 0: distribuição mais pontiaguda (leptocúrtica)
  - Valor < 0: distribuição mais achatada (platicúrtica)

## Distribuição dos dados

Atravé sda visualização dos resultados acima, podemos dizer que todas as colunas possui uma assimetria enorme, contendo uma esparsidade dos dados enorme.

### Scatterplot

In [ ]:

def criar_scatterplot(coluna_y):
    """
    Cria um gráfico de dispersão usando Plotly
    """
    if not coluna_y:
        fig = go.Figure()
        fig.update_layout(title="Selecione uma coluna para visualizar")
        return fig
    
    try:
        # Remove valores nulos da coluna selecionada
        dados_limpos = df_economatica_eda[coluna_y].dropna()
        
        if dados_limpos.empty:
            fig = go.Figure()
            fig.update_layout(title="Sem dados disponíveis para esta coluna")
            return fig
        
        # Cria índices correspondentes aos dados válidos
        indices = dados_limpos.index
        
        # Cria figura Plotly
        fig = go.Figure()
        
        # Adiciona scatter plot
        fig.add_trace(go.Scatter(
            x=indices,
            y=dados_limpos.values,
            mode='markers',
            name=coluna_y,
            marker=dict(
                size=6,
                color=GRADIO_PRIMARY,
                line=dict(
                    width=1,
                    color=GRADIO_BORDER
                )
            ),
            hovertemplate=f'<b>Índice</b>: %{{x}}<br><b>{coluna_y}</b>: %{{y:.4f}}<extra></extra>'
        ))
        
        # Layout
        fig.update_layout(
            title=f'Distribuição de {coluna_y} por Índice',
            xaxis_title='Índice',
            yaxis_title=coluna_y,
            height=500,
            showlegend=False,
            hovermode='closest',
            xaxis=dict(
                showgrid=True,
                gridwidth=1,
                gridcolor=GRADIO_BORDER
            ),
            yaxis=dict(
                showgrid=True,
                gridwidth=1,
                gridcolor=GRADIO_BORDER
            )
        )
        
        return fig
        
    except Exception as e:
        fig = go.Figure()
        fig.update_layout(title=f"Erro ao gerar gráfico: {str(e)}")
        return fig

# Interface Gradio com Plotly
with gr.Blocks(title="Análise de Dispersão") as interface_scatterplot:
    
    # Controle superior
    dropdown_y = gr.Dropdown(
        choices=colunas_numericas, 
        label="Selecione a coluna para visualizar (Eixo Y)",
        value=colunas_numericas[0] if colunas_numericas else None,
    )
    
    # Gráfico Plotly
    scatter_plot = gr.Plot(
        value=criar_scatterplot(colunas_numericas[0] if colunas_numericas else None),
        show_label=False
    )
    
    # Event handlers
    dropdown_y.change(
        fn=criar_scatterplot,
        inputs=dropdown_y,
        outputs=scatter_plot
    )

# Lança a interface
interface_scatterplot.launch(share=False, debug=False, inbrowser=False, quiet=True)

Com o Dataframe reordenado, iremos selecionar um range para cada coluna, para que possamos obter os registros traçados como outliers. Nessa etapa iremos selecionar somente os causadores da esparcidade extrema, como mostrado nos histogramas abaixo.

### Histograma

In [ ]:
def criar_histograma(coluna_selecionada, num_bins=20):
    """
    Cria um gráfico Plotly combinando histograma e curva de distribuição normal
    """
    if not coluna_selecionada:
        fig = go.Figure()
        fig.update_layout(title="Selecione uma coluna para visualizar")
        return fig
    
    try:
        # Remove valores nulos
        dados = df_economatica_eda[coluna_selecionada].dropna()
        
        if dados.empty:
            fig = go.Figure()
            fig.update_layout(title="Sem dados disponíveis para esta coluna")
            return fig
        
        # Cria figura Plotly
        fig = go.Figure()
        
        # Calcula os bins do histograma primeiro para obter a escala correta
        counts, bin_edges = np.histogram(dados, bins=num_bins)
        bin_width = bin_edges[1] - bin_edges[0]
        
        # Adiciona histograma
        fig.add_trace(go.Histogram(
            x=dados,
            nbinsx=num_bins,
            name='Frequência',
            marker_color=GRADIO_PRIMARY,
            marker_line_color=GRADIO_BORDER,
            marker_line_width=1
        ))
        
        # Calcula estatísticas da distribuição
        mu = dados.mean()
        sigma = dados.std()
        
        # Cria pontos para a curva normal
        x_min, x_max = dados.min(), dados.max()
        x_curva = np.linspace(x_min, x_max, 200)
        y_curva_norm = stats.norm.pdf(x_curva, mu, sigma)
        
        # Escala a curva normal para corresponder ao histograma
        # Multiplica pela área total do histograma (número de observações * largura do bin)
        y_curva_scaled = y_curva_norm * len(dados) * bin_width
        
        # Adiciona curva normal escalada
        fig.add_trace(go.Scatter(
            x=x_curva,
            y=y_curva_scaled,
            mode='lines',
            name='Curva Normal',
            line=dict(color=GRADIO_SECONDARY, width=3)
        ))
        
        # Layout usando tema configurado globalmente
        fig.update_layout(
            title=f'Distribuição de {coluna_selecionada}',
            xaxis_title='Valores',
            yaxis_title='Frequência',
            height=500,
            showlegend=True,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="center",
                x=0.5
            )
        )
        
        return fig
        
    except Exception as e:
        # Se der erro, retorna figura vazia com mensagem
        fig = go.Figure()
        fig.update_layout(title=f"Erro ao gerar gráfico: {str(e)}")
        return fig

def resetar_bins():
    """Retorna valor padrão para bins"""
    return 20

# Interface Gradio Simplificada
with gr.Blocks(title="Análise de Distribuição") as interface_histograma:
    
    # Controles superiores
    with gr.Row():
        dropdown_coluna = gr.Dropdown(
            choices=colunas_numericas,
            label="Selecione a Variável",
            value=colunas_numericas[0] if colunas_numericas else None,
            scale=2,
        )
        
        slider_bins = gr.Slider(
            minimum=3,
            maximum=50,
            value=20,
            step=1,
            label="Número de Bins",
            scale=1,
        )
    
    # Gráfico
    histograma = gr.Plot(
        value=criar_histograma(colunas_numericas[0] if colunas_numericas else None, 20),
        show_label=False
    )
    
    # Event handlers
    dropdown_coluna.change(
        fn=lambda col, bins: criar_histograma(col, bins),
        inputs=[dropdown_coluna, slider_bins],
        outputs=histograma
    )
    
    slider_bins.change(
        fn=lambda col, bins: criar_histograma(col, bins),
        inputs=[dropdown_coluna, slider_bins],
        outputs=histograma
    )
    
# Lança a interface
interface_histograma.launch(share=False, debug=False, inbrowser=False, quiet=True)

### Boxplot

In [ ]:
def criar_boxplot(coluna_y, mostrar_outliers=True, escala_log=False):
    """
    Cria um gráfico boxplot usando Plotly
    
    Args:
        coluna_y (str): Nome da coluna para visualizar no eixo Y
        mostrar_outliers (bool): Se True, mostra os outliers no boxplot
        escala_log (bool): Se True, usa escala logarítmica no eixo Y
        
    Returns:
        go.Figure: Figura Plotly com o boxplot
    """
    if not coluna_y:
        fig = go.Figure()
        fig.update_layout(title="Selecione uma coluna para visualizar")
        return fig
    
    try:
        # Remove valores nulos da coluna selecionada
        dados_limpos = df_economatica_eda[coluna_y].dropna()
        
        if dados_limpos.empty:
            fig = go.Figure()
            fig.update_layout(title="Sem dados disponíveis para esta coluna")
            return fig
        
        # Verificar se os dados são compatíveis com escala logarítmica
        tem_valores_negativos = (dados_limpos <= 0).any()
        escala_incompativel = False
        
        if escala_log and tem_valores_negativos:
            escala_incompativel = True
            escala_log = False
        
        # Agrupar por ano para comparar distribuição por ano
        df_agrupado = df_economatica_eda.dropna(subset=[coluna_y])
        anos = sorted(df_agrupado['ano'].unique())
        
        # Cria figura Plotly
        fig = go.Figure()
        
        # Adiciona um boxplot para cada ano
        for ano in anos:
            dados_ano = df_agrupado[df_agrupado['ano'] == ano][coluna_y]
            
            if not dados_ano.empty:
                fig.add_trace(go.Box(
                    y=dados_ano,
                    name=f'Ano {ano}',
                    boxpoints='outliers' if mostrar_outliers else False,
                    jitter=0.3,
                    pointpos=-1.8,
                    marker=dict(
                        color=GRADIO_PRIMARY,
                        line=dict(width=1, color=GRADIO_BORDER)
                    ),
                    line=dict(color=GRADIO_SECONDARY),
                    fillcolor='rgba(255, 255, 255, 0.1)',
                    hoverinfo='y+name',
                    hovertemplate=f'<b>Ano</b>: %{{name}}<br><b>{coluna_y}</b>: %{{y:.4f}}<extra></extra>'
                ))
        
        # Layout usando tema configurado globalmente
        titulo = f'Boxplot de {coluna_y} por Ano'
        if escala_log:
            titulo += " (Escala Log)"
        elif escala_incompativel:
            titulo += " ⚠️ Escala Log não aplicável: dados contêm valores ≤ 0"
            
        fig.update_layout(
            title=titulo,
            yaxis_title=coluna_y,
            height=600,
            showlegend=False,
            boxmode='group',
            boxgap=0.1,
            boxgroupgap=0.2,
            yaxis=dict(
                showgrid=True,
                gridwidth=1,
                gridcolor=GRADIO_BORDER,
                zeroline=True,
                zerolinecolor=GRADIO_BORDER,
                zerolinewidth=1,
                type='log' if escala_log else 'linear'
            )
        )
        
        return fig
        
    except Exception as e:
        fig = go.Figure()
        fig.update_layout(title=f"Erro ao gerar gráfico: {str(e)}")
        return fig

# Interface Gradio com Plotly - com layout melhorado
with gr.Blocks(title="Análise de Boxplot") as interface_boxplot:
    
    # Todos os controles na mesma linha, alinhados verticalmente
    with gr.Row(equal_height=True, variant="panel"):
        with gr.Column(scale=2):
            dropdown_y = gr.Dropdown(
                choices=colunas_numericas, 
                label="Selecione a coluna para visualizar",
                value=colunas_numericas[0] if colunas_numericas else None,
            )
            
        with gr.Column(scale=1):
            mostrar_outliers = gr.Checkbox(
                label="Mostrar Outliers",
                value=True,
            )
            
            escala_log = gr.Checkbox(
                label="Escala Logarítmica",
                value=False,
            )
    
    # Gráfico Plotly
    boxplot_plot = gr.Plot(
        value=criar_boxplot(
            colunas_numericas[0] if colunas_numericas else None,
            True,
            False
        ),
        show_label=False
    )
    
    # Event handlers
    dropdown_y.change(
        fn=criar_boxplot,
        inputs=[dropdown_y, mostrar_outliers, escala_log],
        outputs=boxplot_plot
    )
    
    mostrar_outliers.change(
        fn=criar_boxplot,
        inputs=[dropdown_y, mostrar_outliers, escala_log],
        outputs=boxplot_plot
    )
    
    escala_log.change(
        fn=criar_boxplot,
        inputs=[dropdown_y, mostrar_outliers, escala_log],
        outputs=boxplot_plot
    )

# Lança a interface
interface_boxplot.launch(share=False, debug=False, inbrowser=False, quiet=True)

In [ ]:
def identificar_outliers_consolidados(df):
    """
    Identifica outliers em todas as colunas numéricas usando o método IQR
    e retorna um DataFrame consolidado com todos os outliers.
    
    Args:
        df (pd.DataFrame): DataFrame com os dados
        
    Returns:
        pd.DataFrame: DataFrame consolidado com todos os outliers
    """
    # Lista para armazenar todos os outliers
    outliers_list = []
    
    # Identifica colunas numéricas (excluindo identificadores e categóricas)
    colunas_numericas = [col for col in df.columns 
                        if col not in ['nome', 'codigo', 'setor', 'pandemia', 'ano'] 
                        and df[col].dtype in ['int64', 'float64', 'int32', 'float32']]
    
    print(f"Analisando outliers em {len(colunas_numericas)} colunas numéricas...")
    
    # Para cada coluna numérica, identifica outliers
    for coluna in colunas_numericas:
        # Calcula quartis e IQR
        Q1 = df[coluna].quantile(0.25)
        Q3 = df[coluna].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define limites para outliers
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        
        # Identifica registros que são outliers
        outliers_mask = (df[coluna] < limite_inferior) | (df[coluna] > limite_superior)
        outliers_df = df[outliers_mask].copy()
        
        if not outliers_df.empty:
            # Para cada outlier, adiciona à lista
            for idx, row in outliers_df.iterrows():
                outliers_list.append({
                    'codigo': row['codigo'] if 'codigo' in df.columns else None,
                    'nome': row['nome'] if 'nome' in df.columns else None,
                    'setor': row['setor'] if 'setor' in df.columns else None,
                    'ano': row['ano'] if 'ano' in df.columns else None,
                    'variavel': coluna,
                    'valor': row[coluna],
                    'limite_inferior': limite_inferior,
                    'limite_superior': limite_superior,
                    'is_baixo': row[coluna] < limite_inferior,
                    'is_alto': row[coluna] > limite_superior
                })
    
    # Cria DataFrame com todos os outliers
    outliers_consolidados = pd.DataFrame(outliers_list)
    
    print(f"Total de outliers identificados: {len(outliers_consolidados)}")
    print(f"Em {outliers_consolidados['variavel'].nunique()} variáveis diferentes")
    if 'codigo' in df.columns:
        print(f"Afetando {outliers_consolidados['codigo'].nunique()} códigos únicos")
    
    return outliers_consolidados

In [ ]:
# Cria o DataFrame de outliers
df_economatica_outliers = identificar_outliers_consolidados(df_economatica_eda)

# Mostra resumo dos outliers por variável
print("\n=== RESUMO DOS OUTLIERS POR VARIÁVEL ===")
resumo_outliers = df_economatica_outliers.groupby('variavel').size().sort_values(ascending=False)
print(resumo_outliers)

# Verifica quais empresas têm mais outliers
if 'codigo' in df_economatica_eda.columns:
    print("\n=== EMPRESAS COM MAIS OUTLIERS ===")
    top_empresas = df_economatica_outliers['codigo'].value_counts().head(10)
    print(top_empresas)

É, dado a quantidade de registros que temos, o ideal é não tratar mesmo...

## Insights

# Pré-processamento dos dados

## Normalização dos nomes das colunas

Criado funções para resolver problemas de duplicação de colunas e padronização nos nomes das colunas, serão utilizadas em breve.

In [ ]:
def slugify(txt: str) -> str:
    """
    Converte uma string para um formato "slug", amigável para nomes de coluna.
    Remove acentos, substitui caracteres não alfanuméricos por underscores
    e limpa underscores extras.

    Args:
        txt (str): A string de entrada.

    Returns:
        str: A string convertida para formato slug, ou "missing" se a entrada for nula ou vazia após a limpeza.
    """
    if txt is None:
        return "missing"
    txt = f'setor_{str(txt).lower().strip()}'
    # remover acentos
    txt = unicodedata.normalize("NFKD", txt).encode("ascii", "ignore").decode("ascii")
    # trocar não-alfanuméricos por "_"
    txt = re.sub(r"[^a-z0-9]+", "_", txt)
    # limpar underscores duplicados e bordas
    txt = re.sub(r"_+", "_", txt).strip("_")
    return txt or "missing"

## Gera CSV com os setores para agrupamento manual

In [ ]:
# Aplica slugify na coluna setor ANTES de filtrar e gerar o CSV
df_economatica_sem_nulos_slugified = df_economatica_sem_nulos.copy()
df_economatica_sem_nulos_slugified['setor'] = df_economatica_sem_nulos_slugified['setor'].apply(slugify)

# Filtra dados de 2020 e extrai setores únicos (agora já com slugify aplicado)
df_setores_2020 = df_economatica_sem_nulos_slugified[df_economatica_sem_nulos_slugified['ano'] == 2020][['setor']].drop_duplicates()

# Define o caminho baseado no ambiente
if is_colab:
    caminho_arquivo = '/content/setores_economatica_2020.csv'
else:
    os.makedirs('../datasets', exist_ok=True)
    caminho_arquivo = '../datasets/setores_economatica_2020.csv'

# Exporta o CSV
df_setores_2020.to_csv(caminho_arquivo, index=False, encoding='utf-8')

print(f"✅ CSV exportado: {caminho_arquivo}")
print(f"📊 {len(df_setores_2020)} empresas | {df_setores_2020['setor'].nunique()} setores únicos")
print(f"\nPrimeiras linhas:")
print(df_setores_2020.head())

## Upa CSV com os setores agrupados e substitui os setores através do mapeamento dos grupos

In [ ]:
# URL do CSV com o mapeamento dos setores
url_mapeamento = 'https://drive.google.com/file/d/1ITrtp7GsfZi9eZVTDQgro8fMu23zZu0w/view' 

# Define o caminho baseado no ambiente
if is_colab:
    output_mapeamento = '/content/mapeamento_setores.csv'
else:
    output_mapeamento = '../datasets/mapeamento_setores.csv'

# Baixa o arquivo de mapeamento
print(f"Baixando mapeamento de setores para: {output_mapeamento}")
gdown.download(url_mapeamento, output_mapeamento, fuzzy=True)

# Carrega o CSV de mapeamento
df_mapeamento = pd.read_csv(output_mapeamento, encoding='utf-8')

In [ ]:
print("CSV de mapeamento carregado:")
print(df_mapeamento.head())
print(f"\nShape: {df_mapeamento.shape}")
print(f"Colunas: {list(df_mapeamento.columns)}")

# Cria o dicionário de mapeamento setor -> grupo
mapeamento_setores = dict(zip(df_mapeamento['setor'], df_mapeamento['grupo']))

print(f"\nMapeamento criado com {len(mapeamento_setores)} setores:")
for setor, grupo in list(mapeamento_setores.items())[:5]:  # Mostra apenas os primeiros 5
    print(f"  {setor} -> {grupo}")

In [ ]:
# Aplica slugify na coluna setor do DataFrame principal antes do mapeamento
df_pre_processado = df_economatica_sem_nulos.copy()

df_pre_processado['setor'] = df_pre_processado['setor'].apply(slugify)

df_pre_processado.head(50)

In [ ]:
# Aplica o mapeamento no DataFrame principal
df_pre_processado['setor'] = df_pre_processado['setor'].map(mapeamento_setores)

df_pre_processado.head(50)

In [ ]:
# Verifica se há setores não mapeados (NaN)
setores_nao_mapeados = df_pre_processado['setor'].isnull().sum()
if setores_nao_mapeados > 0:
    print(f"\n⚠️ Atenção: {setores_nao_mapeados} registros com setores não mapeados")
    setores_perdidos = df_pre_processado[df_pre_processado['setor'].isnull()]['codigo'].unique()
    print(f"Códigos afetados: {setores_perdidos[:5]}...")
else:
    print("\n✅ Todos os setores foram mapeados com sucesso!")

# Mostra a distribuição dos novos grupos
print(f"\nDistribuição dos grupos após mapeamento:")
print(df_pre_processado['setor'].value_counts())

print(f"\nTotal de grupos únicos: {df_pre_processado['setor'].nunique()}")

## Fazendo o label encoding

In [ ]:
# Inicia o label encoder
le = LabelEncoder()

# Executa o label encoder na coluna setor
df_pre_processado['setor'] = le.fit_transform(df_pre_processado['setor'])
df_pre_processado['codigo'] = le.fit_transform(df_pre_processado['codigo'])

print("Category Mapping:", le.classes_)

In [ ]:
df_pre_processado.head(50)

## Removendo a coluna nome, codigo e ano

In [ ]:
df_pre_processado.drop(columns=['nome', 'codigo', 'ano'], inplace=True)

## Normalizando os dados

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

def aplicar_minmax_normalizacao(df):
    """
    Aplica MinMaxScaler (0-1) para todas as features numéricas.
    Preserva colunas categóricas e identificadoras.
    """
    print("=== NORMALIZAÇÃO MINMAX (0-1) ===\n")
    
    df_normalizado = df.copy()
    
    # Identifica colunas numéricas para normalizar
    colunas_numericas = [col for col in df.columns 
                        if col not in ['setor', 'ano', 'pandemia', 'variacao_valor_mercado'] 
                        and df[col].dtype in ['int64', 'float64', 'int32', 'float32']]
    
    print(f"🔧 Normalizando {len(colunas_numericas)} features numéricas...")
    
    # Aplica MinMaxScaler
    scaler = MinMaxScaler()
    
    # Transforma apenas as colunas numéricas
    df_normalizado[colunas_numericas] = scaler.fit_transform(df_normalizado[colunas_numericas])
    
    print(f"✅ Normalização concluída!")
    print(f"   - Todas as features estão na escala [0, 1]")
    print(f"   - Colunas preservadas: setor, ano, pandemia, variacao_valor_mercado")
    
    # Estatísticas pós-normalização
    print(f"\n📊 VERIFICAÇÃO PÓS-NORMALIZAÇÃO:")
    stats_pos = df_normalizado[colunas_numericas].describe()
    print(f"   - Valor mínimo global: {stats_pos.loc['min'].min():.6f}")
    print(f"   - Valor máximo global: {stats_pos.loc['max'].max():.6f}")
    print(f"   - Média das médias: {stats_pos.loc['mean'].mean():.6f}")
    
    # Mostra amostra das primeiras features normalizadas
    print(f"\n👀 AMOSTRA DAS PRIMEIRAS 5 FEATURES NORMALIZADAS:")
    for col in colunas_numericas[:5]:
        valores = df_normalizado[col]
        print(f"   - {col}: min={valores.min():.3f}, max={valores.max():.3f}, mean={valores.mean():.3f}")
    
    return df_normalizado, scaler, colunas_numericas

def comparar_antes_depois_minmax(df_original, df_normalizado, colunas_numericas):
    """
    Compara estatísticas antes e depois da normalização MinMax.
    """
    print("\n=== COMPARAÇÃO ANTES vs DEPOIS ===\n")
    
    print(f"{'Feature':<20} {'Antes_Min':<12} {'Antes_Max':<12} {'Depois_Min':<12} {'Depois_Max':<12}")
    print("-" * 80)
    
    for col in colunas_numericas[:10]:  # Mostra apenas as primeiras 10
        antes_min = df_original[col].min()
        antes_max = df_original[col].max()
        depois_min = df_normalizado[col].min()
        depois_max = df_normalizado[col].max()
        
        print(f"{col:<20} {antes_min:<12.2f} {antes_max:<12.2f} {depois_min:<12.6f} {depois_max:<12.6f}")
    
    print(f"\n✅ Todas as features foram normalizadas para a escala [0, 1]")

# Aplicar normalização MinMax
print("=== APLICANDO NORMALIZAÇÃO MINMAX ===\n")

# Aplicar MinMaxScaler
df_pre_processado_normalizado, scaler_usado, features_normalizadas = aplicar_minmax_normalizacao(df_pre_processado)

# Comparar antes e depois
comparar_antes_depois_minmax(df_pre_processado, df_pre_processado_normalizado, features_normalizadas)

# Verificação final
print(f"\n📋 RESUMO FINAL:")
print(f"   - Shape do dataset: {df_pre_processado_normalizado.shape}")
print(f"   - Features normalizadas: {len(features_normalizadas)}")
print(f"   - Método usado: MinMaxScaler [0, 1]")
print(f"   - Dados prontos para modelagem: ✅")

# Mostra amostra final
print(f"\n👀 AMOSTRA DO DATASET NORMALIZADO:")
print(df_pre_processado_normalizado.head())

df_pre_processado_normalizado['variacao_valor_mercado'].astype(int)

## Gerar o dataset pré-processado

In [ ]:
df_pre_processado_normalizado.head()

In [ ]:
# Gerar CSV do dataset pré-processado e normalizado
print("=== EXPORTAÇÃO DO DATASET PRÉ-PROCESSADO E NORMALIZADO ===\n")

# Define o caminho baseado no ambiente
if is_colab:
    caminho_csv_normalizado = '/content/df_economatica_pre_processado_normalizado.csv'
else:
    os.makedirs('../datasets', exist_ok=True)
    caminho_csv_normalizado = '../datasets/df_economatica_pre_processado_normalizado.csv'

# Exporta o CSV
df_pre_processado_normalizado.to_csv(caminho_csv_normalizado, index=False, encoding='utf-8')

print(f"✅ Dataset pré-processado e normalizado exportado para: {caminho_csv_normalizado}")
print(f"📊 Shape do dataset: {df_pre_processado_normalizado.shape}")
print(f"📅 Período: {df_pre_processado_normalizado['ano'].min()} - {df_pre_processado_normalizado['ano'].max()}")
print(f"🏢 Empresas únicas: {df_pre_processado_normalizado['setor'].nunique()} setores")
print(f"📈 Colunas: {df_pre_processado_normalizado.shape[1]}")

# Resumo das colunas por tipo
print(f"\n📋 RESUMO DAS COLUNAS:")
colunas_por_tipo = {
    'Categóricas': ['setor'],
    'Temporais': ['ano', 'pandemia'],
    'Target': ['variacao_valor_mercado'],
    'Features Normalizadas': [col for col in df_pre_processado_normalizado.columns 
                             if col not in ['setor', 'ano', 'pandemia', 'variacao_valor_mercado']]
}

for tipo, colunas in colunas_por_tipo.items():
    colunas_existentes = [col for col in colunas if col in df_pre_processado_normalizado.columns]
    print(f"  - {tipo}: {len(colunas_existentes)} colunas")
    if tipo == 'Features Normalizadas' and len(colunas_existentes) > 0:
        print(f"    Exemplos: {colunas_existentes[:5]}")

print(f"\n🔍 VERIFICAÇÕES FINAIS:")
print(f"  - Valores nulos: {df_pre_processado_normalizado.isnull().sum().sum()}")
print(f"  - Linhas duplicadas: {df_pre_processado_normalizado.duplicated().sum()}")
print(f"  - Registros por empresa (anos): {df_pre_processado_normalizado.groupby('setor').size().describe()}")

# Estatísticas das features normalizadas
features_normalizadas = [col for col in df_pre_processado_normalizado.columns 
                        if col not in ['setor', 'ano', 'pandemia', 'variacao_valor_mercado']]

if features_normalizadas:
    print(f"\n📊 ESTATÍSTICAS DAS FEATURES NORMALIZADAS:")
    stats_normalizadas = df_pre_processado_normalizado[features_normalizadas].describe()
    print(f"  - Valor mínimo global: {stats_normalizadas.loc['min'].min():.6f}")
    print(f"  - Valor máximo global: {stats_normalizadas.loc['max'].max():.6f}")
    print(f"  - Média global: {stats_normalizadas.loc['mean'].mean():.6f}")
    print(f"  - Desvio padrão médio: {stats_normalizadas.loc['std'].mean():.6f}")

# Distribuição da variável target
print(f"\n🎯 DISTRIBUIÇÃO DA VARIÁVEL TARGET:")
target_dist = df_pre_processado_normalizado['variacao_valor_mercado'].value_counts().sort_index()
print(f"  - Classe 0 (desvalorização): {target_dist.get(0, 0):,} registros")
print(f"  - Classe 1 (valorização): {target_dist.get(1, 0):,} registros")
if len(target_dist) > 0:
    total_validos = target_dist.sum()
    print(f"  - Balanceamento: {(target_dist.get(1, 0) / total_validos * 100):.1f}% valorização")

# Distribuição por ano
print(f"\n📅 DISTRIBUIÇÃO POR ANO:")
distribuicao_ano = df_pre_processado_normalizado['ano'].value_counts().sort_index()
for ano, count in distribuicao_ano.items():
    print(f"  - {ano}: {count:,} registros")

# Mostra amostra dos dados finais
print(f"\n👀 AMOSTRA DOS DADOS FINAIS:")
print(df_pre_processado_normalizado.head())

print(f"\n📁 Arquivo salvo com sucesso!")
print(f"   Caminho: {caminho_csv_normalizado}")
print(f"   Tamanho estimado: ~{df_pre_processado_normalizado.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"   Pronto para modelagem: ✅")

# Informações adicionais para modelagem
print(f"\n🚀 INFORMAÇÕES PARA MODELAGEM:")
print(f"  - Todas as features numéricas normalizadas: [0, 1]")
print(f"  - Setor: Label encoded (categórica)")
print(f"  - Ano: Numérico (2021-2024)")
print(f"  - Pandemia: Binária (0/1)")
print(f"  - Target: Binária (0/1) - Classificação")
print(f"  - Sem valores nulos")
print(f"  - Sem duplicatas")
print(f"  - Dataset balanceado disponível para split treino/teste")